In [1]:
import os
import time
import json
import warnings
import gc
import pickle
import joblib
import glob
import traceback
from dotenv import load_dotenv
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import pandas_ta as ta
from collections import Counter
from numba import jit
from sklearn.cluster import DBSCAN
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.feature_selection import (
    SelectKBest, RFE,
    mutual_info_classif, mutual_info_regression
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
    ,log_loss
)
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import (
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier,
    GradientBoostingClassifier, HistGradientBoostingClassifier,
    RandomForestClassifier, StackingClassifier, VotingClassifier,
    AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor,
    GradientBoostingRegressor, RandomForestRegressor,
    StackingRegressor, VotingRegressor
)
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from lightgbm.callback import early_stopping
from xgboost import XGBClassifier, XGBRegressor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Input, Dense, Flatten, Dropout, Activation,
    LSTM, GRU, SimpleRNN, Bidirectional,
    Conv1D, MaxPooling1D, AveragePooling1D,
    GlobalAveragePooling1D, GlobalMaxPooling1D,
    BatchNormalization, LayerNormalization,
    Attention, MultiHeadAttention,
    Concatenate, Add, Multiply, Lambda,
    Reshape, Permute, RepeatVector, TimeDistributed
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.var_model import VAR
try:
    import torch
    import torch.nn as nn
except ImportError:
    pass
# ============================================================================
# 환경 설정 및 경고 무시
# ============================================================================

# GPU 메모리 증가 허용 설정
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

warnings.filterwarnings('ignore')


DATA_DIR_MAIN = './macro_data'
DATA_DIR_NEW = './macro_data/macro_data'

TRAIN_START_DATE = pd.to_datetime('2020-01-01')
LOOKBACK_DAYS = 200
LOOKBACK_START_DATE = TRAIN_START_DATE - timedelta(days=LOOKBACK_DAYS)


def standardize_date_column(df,file_name):
    """날짜 컬럼 자동 탐지 + datetime 통일 + tz 제거 + 시각 제거"""

    date_cols = [col for col in df.columns if 'date' in col.lower()]
    if not date_cols:
        print("[Warning] 날짜 컬럼을 찾을 수 없습니다.")
        return df
    date_col = date_cols[0]
    
    if date_col != 'date':
        df.rename(columns={date_col: 'date'}, inplace=True)
    

    if file_name == 'eth_onchain.csv':
        df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d', errors='coerce')
    else:
        df['date'] = pd.to_datetime(df['date'], errors='coerce', infer_datetime_format=True)
    
    df = df.dropna(subset=['date'])
    df['date'] = df['date'].dt.normalize()  
    if pd.api.types.is_datetime64tz_dtype(df['date']):
        df['date'] = df['date'].dt.tz_convert(None)
    else:
        df['date'] = df['date'].dt.tz_localize(None)
    print(df.shape,file_name)
    return df


def load_csv(directory, filename):
    filepath = os.path.join(directory, filename)
    if not os.path.exists(filepath):
        print(f"[Warning] {filename} not found")
        return pd.DataFrame()
    df = pd.read_csv(filepath)
    return standardize_date_column(df, filename)


def add_prefix(df, prefix):
    if df.empty:
        return df
    df.columns = [f"{prefix}_{col}" if col != 'date' else col for col in df.columns]
    return df


def create_sentiment_features(news_df):
    if news_df.empty:
        return pd.DataFrame(columns=['date'])
    
    agg = news_df.groupby('date').agg(
        sentiment_mean=('label', 'mean'),
        sentiment_std=('label', 'std'),
        news_count=('label', 'count'),
        positive_ratio=('label', lambda x: (x == 1).sum() / len(x)),
        negative_ratio=('label', lambda x: (x == -1).sum() / len(x)),
        extreme_positive_count=('label', lambda x: (x == 1).sum()),
        extreme_negative_count=('label', lambda x: (x == -1).sum()),
        sentiment_sum=('label', 'sum'),
    ).reset_index().fillna(0)
    
    agg['sentiment_polarity'] = agg['positive_ratio'] - agg['negative_ratio']
    agg['sentiment_intensity'] = agg['positive_ratio'] + agg['negative_ratio']
    agg['sentiment_disagreement'] = agg['positive_ratio'] * agg['negative_ratio']
    agg['bull_bear_ratio'] = agg['positive_ratio'] / (agg['negative_ratio'] + 1e-10)
    agg['weighted_sentiment'] = agg['sentiment_mean'] * np.log1p(agg['news_count'])
    agg['extremity_index'] = (agg['extreme_positive_count'] + agg['extreme_negative_count']) / (agg['news_count'] + 1e-10)
    
    for window in [3,7]:
        agg[f'sentiment_ma{window}'] = agg['sentiment_mean'].rolling(window=window, min_periods=1).mean()
        agg[f'sentiment_volatility_{window}'] = agg['sentiment_mean'].rolling(window=window, min_periods=1).std()
    
    agg['sentiment_trend'] = agg['sentiment_mean'].diff()
    agg['sentiment_acceleration'] = agg['sentiment_trend'].diff()
    agg['news_volume_change'] = agg['news_count'].pct_change()
    
    for window in [7, 14]:
        agg[f'news_volume_ma{window}'] = agg['news_count'].rolling(window=window, min_periods=1).mean()
    
    return agg.fillna(0)


def smart_fill_missing(df_merged):
    REFERENCE_START_DATE = pd.to_datetime('2020-01-01')
    
    for col in df_merged.columns:
        if col == 'date':
            continue
        
        if df_merged[col].isnull().sum() == 0:
            continue
        
        non_null_idx = df_merged[col].first_valid_index()
        
        if non_null_idx is None:
            df_merged[col] = df_merged[col].fillna(0)
            continue
        
        first_date = df_merged.loc[non_null_idx, 'date']
        
        before_mask = df_merged['date'] < first_date
        after_mask = df_merged['date'] >= first_date
        
        df_merged.loc[before_mask, col] = df_merged.loc[before_mask, col].fillna(0)
        df_merged.loc[after_mask, col] = df_merged.loc[after_mask, col].fillna(method='ffill')
        
        remaining = df_merged.loc[after_mask, col].isnull().sum()
        if remaining > 0:
            df_merged.loc[after_mask, col] = df_merged.loc[after_mask, col].fillna(0)
    
    return df_merged


print("="*80)
print("DATA LOADING")
print("="*80)

#news_df = load_csv(DATA_DIR_MAIN, 'news_data.csv')
#eth_onchain_df = load_csv(DATA_DIR_MAIN, 'eth_onchain.csv')
macro_df = load_csv(DATA_DIR_NEW, 'macro_crypto_data.csv')
sp500_df = load_csv(DATA_DIR_NEW, 'SP500.csv')
vix_df = load_csv(DATA_DIR_NEW, 'VIX.csv')
gold_df = load_csv(DATA_DIR_NEW, 'GOLD.csv')
dxy_df = load_csv(DATA_DIR_NEW, 'DXY.csv')
fear_greed_df = load_csv(DATA_DIR_NEW, 'fear_greed.csv')
eth_funding_df = load_csv(DATA_DIR_NEW, 'eth_funding_rate.csv')
usdt_eth_mcap_df = load_csv(DATA_DIR_NEW, 'usdt_eth_mcap.csv')
aave_tvl_df = load_csv(DATA_DIR_NEW, 'aave_eth_tvl.csv')
lido_tvl_df = load_csv(DATA_DIR_NEW, 'lido_eth_tvl.csv')
makerdao_tvl_df = load_csv(DATA_DIR_NEW, 'makerdao_eth_tvl.csv')
uniswap_tvl_df = load_csv(DATA_DIR_NEW, 'uniswap_eth_tvl.csv')
curve_tvl_df = load_csv(DATA_DIR_NEW, 'curve-dex_eth_tvl.csv')
eth_chain_tvl_df = load_csv(DATA_DIR_NEW, 'eth_chain_tvl.csv')
layer2_tvl_df = load_csv(DATA_DIR_NEW, 'layer2_tvl.csv')

print(f"Loaded {len([df for df in [fear_greed_df, eth_funding_df, usdt_eth_mcap_df, aave_tvl_df, lido_tvl_df, makerdao_tvl_df, uniswap_tvl_df, curve_tvl_df, eth_chain_tvl_df, layer2_tvl_df] if not df.empty])} files")

all_dataframes = [
    macro_df, fear_greed_df, usdt_eth_mcap_df,
    aave_tvl_df, lido_tvl_df, makerdao_tvl_df, uniswap_tvl_df, curve_tvl_df,
    eth_chain_tvl_df, eth_funding_df, layer2_tvl_df, 
    sp500_df, vix_df, gold_df, dxy_df#,news_df, eth_onchain_df
]

last_dates = [
    pd.to_datetime(df['date']).max() 
    for df in all_dataframes 
    if not df.empty and 'date' in df.columns
]

end_date = min(last_dates) if last_dates else pd.Timestamp.today()

print("\n" + "="*80)
print("SENTIMENT FEATURES")
print("="*80)
#sentiment_features = create_sentiment_features(news_df)
#print(f"Generated {sentiment_features.shape[1]-1} features")

print("\n" + "="*80)
print("DATA MERGING")
print("="*80)

#eth_onchain_df = add_prefix(eth_onchain_df, 'eth')
fear_greed_df = add_prefix(fear_greed_df, 'fg')
usdt_eth_mcap_df = add_prefix(usdt_eth_mcap_df, 'usdt')
aave_tvl_df = add_prefix(aave_tvl_df, 'aave')
lido_tvl_df = add_prefix(lido_tvl_df, 'lido')
makerdao_tvl_df = add_prefix(makerdao_tvl_df, 'makerdao')
uniswap_tvl_df = add_prefix(uniswap_tvl_df, 'uniswap')
curve_tvl_df = add_prefix(curve_tvl_df, 'curve')
eth_chain_tvl_df = add_prefix(eth_chain_tvl_df, 'chain')
eth_funding_df = add_prefix(eth_funding_df, 'funding')
layer2_tvl_df = add_prefix(layer2_tvl_df, 'l2')
sp500_df = add_prefix(sp500_df, 'sp500')
vix_df = add_prefix(vix_df, 'vix')
gold_df = add_prefix(gold_df, 'gold')
dxy_df = add_prefix(dxy_df, 'dxy')

date_range = pd.date_range(start=LOOKBACK_START_DATE, end=end_date, freq='D')
df_merged = pd.DataFrame(date_range, columns=['date'])

dataframes_to_merge = [
    macro_df,  fear_greed_df, usdt_eth_mcap_df,
    aave_tvl_df, lido_tvl_df, makerdao_tvl_df, uniswap_tvl_df, curve_tvl_df,
    eth_chain_tvl_df, eth_funding_df, layer2_tvl_df,
    sp500_df, vix_df, gold_df, dxy_df#,sentiment_features,eth_onchain_df,
]

for df in dataframes_to_merge:
    if not df.empty:
        df_merged = pd.merge(df_merged, df, on='date', how='left')

print(f"Merged shape: {df_merged.shape}")
print(f"Missing before fill: {df_merged.isnull().sum().sum():,}")

print("\n" + "="*80)
print("MISSING VALUE HANDLING")
print("="*80)

df_merged = smart_fill_missing(df_merged)

missing_after = df_merged.isnull().sum().sum()
print(f"Missing after fill: {missing_after:,}")

if missing_after > 0:
    df_merged = df_merged.fillna(0)
    print(f"Remaining filled with 0")

lookback_df = df_merged[df_merged['date'] < TRAIN_START_DATE]
cols_to_drop = [
    col for col in lookback_df.columns 
    if lookback_df[col].isnull().all() and col != 'date'
]

if cols_to_drop:
    print(f"\nDropping {len(cols_to_drop)} fully missing columns")
    df_merged = df_merged.drop(columns=cols_to_drop)

print(f"Shape: {df_merged.shape}")
print(f"Period: {df_merged['date'].min().date()} ~ {df_merged['date'].max().date()}")
print(f"Missing: {df_merged.isnull().sum().sum()}")


2025-11-19 23:24:58.912222: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-19 23:24:58.912268: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-19 23:24:58.913655: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-19 23:24:58.920709: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-19 23:25:00.128363: W tensorflow/comp

DATA LOADING
(2978, 41) macro_crypto_data.csv
(2233, 2) SP500.csv
(2234, 2) VIX.csv
(2235, 2) GOLD.csv
(2236, 2) DXY.csv
(2845, 2) fear_greed.csv
(2185, 2) eth_funding_rate.csv
(2913, 6) usdt_eth_mcap.csv
(2009, 2) aave_eth_tvl.csv
(1795, 2) lido_eth_tvl.csv
(2511, 2) makerdao_eth_tvl.csv
(2570, 2) uniswap_eth_tvl.csv
(2103, 2) curve-dex_eth_tvl.csv
(2974, 2) eth_chain_tvl.csv
(1603, 5) layer2_tvl.csv
Loaded 10 files

SENTIMENT FEATURES

DATA MERGING
Merged shape: (2346, 62)
Missing before fill: 23,712

MISSING VALUE HANDLING
Missing after fill: 0
Shape: (2346, 62)
Period: 2019-06-15 ~ 2025-11-14
Missing: 0


In [2]:
def add_indicator_to_df(df_ta, indicator):
    """pandas_ta 지표 결과를 DataFrame에 안전하게 추가"""
    if indicator is None:
        return

    if isinstance(indicator, pd.DataFrame) and not indicator.empty:
        for col in indicator.columns:
            df_ta[col] = indicator[col]
    elif isinstance(indicator, pd.Series) and not indicator.empty:
        colname = indicator.name if indicator.name else 'Unnamed'
        df_ta[colname] = indicator

def safe_add(df_ta, func, *args, **kwargs):
    """지표 생성 시 오류 방지를 위한 래퍼 함수"""
    try:
        result = func(*args, **kwargs)
        add_indicator_to_df(df_ta, result)
        return True
    except Exception as e:
        func_name = func.__name__ if hasattr(func, '__name__') else str(func)
        print(f"    ⚠ {func_name.upper()} 생성 실패: {str(e)[:50]}")
        return False

def calculate_technical_indicators(df):
    df = df.sort_values('date').reset_index(drop=True)
    df_ta = df.copy()

    close = df['ETH_Close']
    high = df.get('ETH_High', close)
    low = df.get('ETH_Low', close)
    volume = df.get('ETH_Volume', pd.Series(index=df.index, data=1))
    open_ = df.get('ETH_Open', close)

    try:
        df_ta['RSI_14'] = ta.rsi(close, length=14)
        safe_add(df_ta, ta.macd, close, fast=12, slow=26, signal=9)
        safe_add(df_ta, ta.stoch, high, low, close, k=14, d=3)
        df_ta['WILLR_14'] = ta.willr(high, low, close, length=14)
        df_ta['ROC_10'] = ta.roc(close, length=10)
        df_ta['MOM_10'] = ta.mom(close, length=10)
        df_ta['CCI_14'] = ta.cci(high, low, close, length=14)
        df_ta['CCI_50'] = ta.cci(high, low, close, length=50)
        df_ta['CCI_SIGNAL'] = (df_ta['CCI_14'] > 100).astype(int)
        safe_add(df_ta, ta.tsi, close, fast=13, slow=25, signal=13)

        try:
            ichimoku = ta.ichimoku(high, low, close)
            if ichimoku is not None and isinstance(ichimoku, tuple):
                ichimoku_df = ichimoku[0]
                if ichimoku_df is not None:
                    for col in ichimoku_df.columns:
                        df_ta[col] = ichimoku_df[col]
        except:
            pass

        df_ta['SMA_20'] = ta.sma(close, length=20)
        df_ta['SMA_50'] = ta.sma(close, length=50)
        df_ta['EMA_12'] = ta.ema(close, length=12)
        df_ta['EMA_26'] = ta.ema(close, length=26)
        df_ta['TEMA_10'] = ta.tema(close, length=10)
        df_ta['WMA_20'] = ta.wma(close, length=20)
        df_ta['HMA_9'] = ta.hma(close, length=9)
        df_ta['DEMA_10'] = ta.dema(close, length=10)
        df_ta['VWMA_20'] = ta.vwma(close, volume, length=20)
        df_ta['HL2'] = ta.hl2(high, low)
        df_ta['HLC3'] = ta.hlc3(high, low, close)
        df_ta['OHLC4'] = ta.ohlc4(open_, high, low, close)

        safe_add(df_ta, ta.bbands, close, length=20, std=2)
        df_ta['ATR_14'] = ta.atr(high, low, close, length=14)
        df_ta['NATR_14'] = ta.natr(high, low, close, length=14)

        try:
            tr = ta.true_range(high, low, close)
            if isinstance(tr, pd.Series) and not tr.empty:
                df_ta['TRUERANGE'] = tr
            elif isinstance(tr, pd.DataFrame) and not tr.empty:
                df_ta['TRUERANGE'] = tr.iloc[:, 0]
        except:
            pass

        safe_add(df_ta, ta.kc, high, low, close, length=20)

        try:
            dc = ta.donchian(high, low, lower_length=20, upper_length=20)
            if dc is not None and isinstance(dc, pd.DataFrame) and not dc.empty:
                for col in dc.columns:
                    df_ta[col] = dc[col]
        except:
            pass

        atr_10 = ta.atr(high, low, close, length=10)
        hl2_calc = (high + low) / 2
        upper_band = hl2_calc + (3 * atr_10)
        lower_band = hl2_calc - (3 * atr_10)

        df_ta['SUPERTREND'] = 0
        for i in range(1, len(df_ta)):
            if close.iloc[i] > upper_band.iloc[i-1]:
                df_ta.loc[df_ta.index[i], 'SUPERTREND'] = 1
            elif close.iloc[i] < lower_band.iloc[i-1]:
                df_ta.loc[df_ta.index[i], 'SUPERTREND'] = -1
            else:
                df_ta.loc[df_ta.index[i], 'SUPERTREND'] = df_ta['SUPERTREND'].iloc[i-1]

        df_ta['OBV'] = ta.obv(close, volume)
        df_ta['AD'] = ta.ad(high, low, close, volume)
        df_ta['ADOSC_3_10'] = ta.adosc(high, low, close, volume, fast=3, slow=10)
        df_ta['MFI_14'] = ta.mfi(high, low, close, volume, length=14)
        df_ta['CMF_20'] = ta.cmf(high, low, close, volume, length=20)
        df_ta['EFI_13'] = ta.efi(close, volume, length=13)
        safe_add(df_ta, ta.eom, high, low, close, volume, length=14)

        try:
            df_ta['VWAP'] = ta.vwap(high, low, close, volume)
        except:
            pass

        safe_add(df_ta, ta.adx, high, low, close, length=14)

        try:
            aroon = ta.aroon(high, low, length=25)
            if aroon is not None and isinstance(aroon, pd.DataFrame):
                for col in aroon.columns:
                    df_ta[col] = aroon[col]
        except:
            pass

        try:
            psar = ta.psar(high, low, close)
            if psar is not None:
                if isinstance(psar, pd.DataFrame) and not psar.empty:
                    for col in psar.columns:
                        df_ta[col] = psar[col]
                elif isinstance(psar, pd.Series) and not psar.empty:
                    df_ta[psar.name] = psar
        except:
            pass

        safe_add(df_ta, ta.vortex, high, low, close, length=14)
        df_ta['DPO_20'] = ta.dpo(close, length=20)

        df_ta['PRICE_CHANGE'] = close.pct_change()
        df_ta['VOLATILITY_20'] = close.pct_change().rolling(window=20).std()
        df_ta['MOMENTUM_10'] = close / close.shift(10) - 1
        df_ta['PRICE_VS_SMA20'] = close / df_ta['SMA_20'] - 1
        df_ta['PRICE_VS_EMA12'] = close / df_ta['EMA_12'] - 1
        df_ta['SMA_GOLDEN_CROSS'] = (df_ta['SMA_50'] > df_ta['SMA_20']).astype(int)
        df_ta['EMA_CROSS_SIGNAL'] = (df_ta['EMA_12'] > df_ta['EMA_26']).astype(int)
        df_ta['VOLUME_SMA_20'] = ta.sma(volume, length=20)
        df_ta['VOLUME_RATIO'] = volume / (df_ta['VOLUME_SMA_20'] + 1e-10)
        df_ta['VOLUME_CHANGE'] = volume.pct_change()
        df_ta['VOLUME_CHANGE_5'] = volume.pct_change(periods=5)
        df_ta['HIGH_LOW_RANGE'] = (high - low) / (close + 1e-10)
        df_ta['HIGH_CLOSE_RANGE'] = np.abs(high - close.shift()) / (close + 1e-10)
        df_ta['CLOSE_LOW_RANGE'] = (close - low) / (close + 1e-10)
        df_ta['INTRADAY_POSITION'] = (close - low) / ((high - low) + 1e-10)

        try:
            df_ta['SLOPE_5'] = ta.linreg(close, length=5, slope=True)
        except:
            df_ta['SLOPE_5'] = close.rolling(window=5).apply(
                lambda x: np.polyfit(np.arange(len(x)), x, 1)[0] if len(x) == 5 else np.nan, raw=True
            )

        df_ta['INC_1'] = (close > close.shift(1)).astype(int)
        df_ta['BOP'] = (close - open_) / ((high - low) + 1e-10)
        df_ta['BOP'] = df_ta['BOP'].fillna(0)

        if 'BBL_20' in df_ta.columns and 'BBU_20' in df_ta.columns and 'BBM_20' in df_ta.columns:
            df_ta['BB_WIDTH'] = (df_ta['BBU_20'] - df_ta['BBL_20']) / (df_ta['BBM_20'] + 1e-8)
            df_ta['BB_POSITION'] = (close - df_ta['BBL_20']) / (df_ta['BBU_20'] - df_ta['BBL_20'] + 1e-8)

        df_ta['RSI_OVERBOUGHT'] = (df_ta['RSI_14'] > 70).astype(int)
        df_ta['RSI_OVERSOLD'] = (df_ta['RSI_14'] < 30).astype(int)

        if 'MACDh_12_26_9' in df_ta.columns:
            df_ta['MACD_HIST_CHANGE'] = df_ta['MACDh_12_26_9'].diff()

        df_ta['VOLUME_STRENGTH'] = volume / volume.rolling(window=50).mean()
        df_ta['PRICE_ACCELERATION'] = close.pct_change().diff()
        df_ta['GAP'] = (open_ - close.shift(1)) / (close.shift(1) + 1e-10)
        df_ta['ROLLING_MAX_20'] = close.rolling(window=20).max()
        df_ta['ROLLING_MIN_20'] = close.rolling(window=20).min()
        df_ta['DISTANCE_FROM_HIGH'] = (df_ta['ROLLING_MAX_20'] - close) / (df_ta['ROLLING_MAX_20'] + 1e-10)
        df_ta['DISTANCE_FROM_LOW'] = (close - df_ta['ROLLING_MIN_20']) / (close + 1e-10)

        ret_squared = close.pct_change() ** 2
        df_ta['RV_5'] = ret_squared.rolling(5).sum()
        df_ta['RV_20'] = ret_squared.rolling(20).sum()
        df_ta['RV_RATIO'] = df_ta['RV_5'] / (df_ta['RV_20'] + 1e-10)

    except Exception:
        pass

    return df_ta



def add_enhanced_cross_crypto_features(df):
    df_enhanced = df.copy()
    df_enhanced['eth_return'] = df['ETH_Close'].pct_change()
    df_enhanced['btc_return'] = df['BTC_Close'].pct_change()

    for lag in [1, 5]:
        df_enhanced[f'btc_return_lag{lag}'] = df_enhanced['btc_return'].shift(lag)

    for window in [3, 7, 14, 30, 60]:
        df_enhanced[f'eth_btc_corr_{window}d'] = (
            df_enhanced['eth_return'].rolling(window).corr(df_enhanced['btc_return'])
        )

    eth_vol = df_enhanced['eth_return'].abs()
    btc_vol = df_enhanced['btc_return'].abs()

    for window in [7, 14, 30]:
        df_enhanced[f'eth_btc_volcorr_{window}d'] = eth_vol.rolling(window).corr(btc_vol)
        df_enhanced[f'eth_btc_volcorr_sq_{window}d'] = (
            (df_enhanced['eth_return']**2).rolling(window).corr(df_enhanced['btc_return']**2)
        )

    df_enhanced['btc_eth_strength_ratio'] = (
        df_enhanced['btc_return'] / (df_enhanced['eth_return'].abs() + 1e-8)
    )
    df_enhanced['btc_eth_strength_ratio_7d'] = df_enhanced['btc_eth_strength_ratio'].rolling(7).mean()

    alt_returns = []
    for coin in ['BNB', 'XRP', 'SOL', 'ADA']:
        if f'{coin}_Close' in df.columns:
            alt_returns.append(df[f'{coin}_Close'].pct_change())

    if alt_returns:
        market_return = pd.concat(
            alt_returns + [df_enhanced['eth_return'], df_enhanced['btc_return']], axis=1
        ).mean(axis=1)
        df_enhanced['btc_dominance'] = df_enhanced['btc_return'] / (market_return + 1e-8)

    for window in [30, 60, 90]:
        covariance = df_enhanced['eth_return'].rolling(window).cov(df_enhanced['btc_return'])
        btc_variance = df_enhanced['btc_return'].rolling(window).var()
        df_enhanced[f'eth_btc_beta_{window}d'] = covariance / (btc_variance + 1e-8)

    df_enhanced['eth_btc_spread'] = df_enhanced['eth_return'] - df_enhanced['btc_return']
    df_enhanced['eth_btc_spread_ma7'] = df_enhanced['eth_btc_spread'].rolling(7).mean()
    df_enhanced['eth_btc_spread_std7'] = df_enhanced['eth_btc_spread'].rolling(7).std()

    btc_vol_ma = btc_vol.rolling(30).mean()
    high_vol_mask = btc_vol > btc_vol_ma
    df_enhanced['eth_btc_corr_highvol'] = np.nan
    df_enhanced['eth_btc_corr_lowvol'] = np.nan

    for i in range(30, len(df_enhanced)):
        window_data = df_enhanced.iloc[i-30:i]
        high_vol_data = window_data[high_vol_mask.iloc[i-30:i]]
        low_vol_data = window_data[~high_vol_mask.iloc[i-30:i]]

        if len(high_vol_data) > 5:
            df_enhanced.loc[df_enhanced.index[i], 'eth_btc_corr_highvol'] = (
                high_vol_data['eth_return'].corr(high_vol_data['btc_return'])
            )
        if len(low_vol_data) > 5:
            df_enhanced.loc[df_enhanced.index[i], 'eth_btc_corr_lowvol'] = (
                low_vol_data['eth_return'].corr(low_vol_data['btc_return'])
            )

    return df_enhanced


def remove_raw_prices_and_transform(df,target_type,method):
    df_transformed = df.copy()

    if 'eth_log_return' not in df_transformed.columns:
        df_transformed['eth_log_return'] = np.log(df['ETH_Close'] / df['ETH_Close'].shift(1))
    if 'eth_intraday_range' not in df_transformed.columns:
        df_transformed['eth_intraday_range'] = (df['ETH_High'] - df['ETH_Low']) / (df['ETH_Close'] + 1e-8)
    if 'eth_body_ratio' not in df_transformed.columns:
        df_transformed['eth_body_ratio'] = (df['ETH_Close'] - df['ETH_Open']) / (df['ETH_Close'] + 1e-8)
    if 'eth_close_position' not in df_transformed.columns:
        df_transformed['eth_close_position'] = (
            (df['ETH_Close'] - df['ETH_Low']) / (df['ETH_High'] - df['ETH_Low'] + 1e-8)
        )

    if 'BTC_Close' in df_transformed.columns:
        for period in [5, 20]:
            col_name = f'btc_return_{period}d'
            if col_name not in df_transformed.columns:
                df_transformed[col_name] = np.log(df['BTC_Close'] / df['BTC_Close'].shift(period)).fillna(0)
        
        for period in [7, 14, 30]:
            col_name = f'btc_volatility_{period}d'
            if col_name not in df_transformed.columns:
                df_transformed[col_name] = (
                    df_transformed['eth_log_return'].rolling(period, min_periods=max(3, period//3)).std()
                ).fillna(0)
        
        if 'btc_intraday_range' not in df_transformed.columns:
            df_transformed['btc_intraday_range'] = (df['BTC_High'] - df['BTC_Low']) / (df['BTC_Close'] + 1e-8)
        if 'btc_body_ratio' not in df_transformed.columns:
            df_transformed['btc_body_ratio'] = (df['BTC_Close'] - df['BTC_Open']) / (df['BTC_Close'] + 1e-8)

        if 'BTC_Volume' in df.columns:
            btc_volume = df['BTC_Volume']
            if 'btc_volume_change' not in df_transformed.columns:
                df_transformed['btc_volume_change'] = btc_volume.pct_change().fillna(0)
            if 'btc_volume_ratio_20d' not in df_transformed.columns:
                volume_ma20 = btc_volume.rolling(20, min_periods=5).mean()
                df_transformed['btc_volume_ratio_20d'] = (btc_volume / (volume_ma20 + 1e-8)).fillna(1)
            if 'btc_volume_volatility_30d' not in df_transformed.columns:
                df_transformed['btc_volume_volatility_30d'] = (
                    btc_volume.pct_change().rolling(30, min_periods=10).std()
                ).fillna(0)
            if 'btc_obv' not in df_transformed.columns:
                btc_close = df['BTC_Close']
                obv = np.where(btc_close > btc_close.shift(1), btc_volume,
                               np.where(btc_close < btc_close.shift(1), -btc_volume, 0))
                df_transformed['btc_obv'] = pd.Series(obv, index=df.index).cumsum().fillna(0)
            if 'btc_volume_price_corr_30d' not in df_transformed.columns:
                df_transformed['btc_volume_price_corr_30d'] = (
                    btc_volume.pct_change().rolling(30, min_periods=10).corr(
                        df_transformed['eth_log_return']
                    )
                ).fillna(0)

    altcoins = ['BNB', 'XRP', 'SOL', 'ADA', 'DOGE', 'AVAX', 'DOT']
    for coin in altcoins:
        if f'{coin}_Close' in df_transformed.columns:
            col_name = f'{coin.lower()}_return'
            if col_name not in df_transformed.columns:
                df_transformed[col_name] = np.log(df[f'{coin}_Close'] / df[f'{coin}_Close'].shift(1)).fillna(0)
            vol_col = f'{coin.lower()}_volatility_30d'
            if vol_col not in df_transformed.columns:
                df_transformed[vol_col] = df_transformed[col_name].rolling(30, min_periods=10).std().fillna(0)
            
            if f'{coin}_Volume' in df.columns:
                coin_volume = df[f'{coin}_Volume']
                volume_change_col = f'{coin.lower()}_volume_change'
                if volume_change_col not in df_transformed.columns:
                    df_transformed[volume_change_col] = coin_volume.pct_change().fillna(0)
                volume_ratio_col = f'{coin.lower()}_volume_ratio_20d'
                if volume_ratio_col not in df_transformed.columns:
                    volume_ma20 = coin_volume.rolling(20, min_periods=5).mean()
                    df_transformed[volume_ratio_col] = (coin_volume / (volume_ma20 + 1e-8)).fillna(1)

    if 'ETH_Volume' in df.columns and 'BTC_Volume' in df.columns:
        eth_volume = df['ETH_Volume']
        btc_volume = df['BTC_Volume']
        if 'eth_btc_volume_corr_30d' not in df_transformed.columns:
            df_transformed['eth_btc_volume_corr_30d'] = (
                eth_volume.pct_change().rolling(30, min_periods=10).corr(btc_volume.pct_change())
            ).fillna(0)
        if 'eth_btc_volume_ratio' not in df_transformed.columns:
            df_transformed['eth_btc_volume_ratio'] = (eth_volume / (btc_volume + 1e-8)).fillna(0)
        if 'eth_btc_volume_ratio_ma30' not in df_transformed.columns:
            df_transformed['eth_btc_volume_ratio_ma30'] = (
                df_transformed['eth_btc_volume_ratio'].rolling(30, min_periods=10).mean()
            ).fillna(0)

            
    ## raw_data 저장하기
    timestamp = datetime.now().strftime("%Y-%m-%d")
    base_dir=os.path.join('model_results',timestamp,'raw_data',target_type,method)
    os.makedirs(base_dir,exist_ok=True)
    df.to_csv(os.path.join(base_dir,"raw_data_all_features.csv"),index=False)        
            
            
    remove_patterns = ['_Close', '_Open', '_High', '_Low', '_Volume']
    cols_to_remove = [
        col for col in df_transformed.columns
        if any(p in col for p in remove_patterns)
        and not any(d in col.lower() for d in ['_lag', '_position', '_ratio', '_range', '_change', '_corr', '_volatility', '_obv'])
    ]
    df_transformed.drop(cols_to_remove, axis=1, inplace=True)

    return_cols = [col for col in df_transformed.columns if 'return' in col.lower() and 'next' not in col]
    if return_cols:
        df_transformed[return_cols] = df_transformed[return_cols].fillna(0)

    return df_transformed

In [3]:
def apply_lag_features(df, news_lag=2, onchain_lag=1):
    df_lagged = df.copy()
    
    raw_sentiment_cols = ['sentiment_mean', 'sentiment_std', 'news_count', 'positive_ratio', 'negative_ratio']
    sentiment_ma_cols = [col for col in df.columns if 'sentiment' in col and ('_ma7' in col or '_volatility_7' in col)]
    no_lag_patterns = ['_trend', '_acceleration', '_volume_change', 'news_volume_change', 'news_volume_ma']
    onchain_cols = [col for col in df.columns if any(keyword in col.lower() 
                    for keyword in ['eth_tx', 'eth_active', 'eth_new', 'eth_large', 'eth_token', 
                                  'eth_contract', 'eth_avg_gas', 'eth_total_gas', 'eth_avg_block'])]
    other_cols = [col for col in df.columns if any(keyword in col.lower() 
                  for keyword in ['tvl', 'funding', 'lido_', 'aave_', 'makerdao_', 
                                'chain_', 'usdt_', 'sp500_', 'vix_', 'gold_', 'dxy_', 'fg_'])]
    
    exclude_cols = ['ETH_Close', 'ETH_High', 'ETH_Low', 'ETH_Open', 'date']
    exclude_cols.extend([col for col in df.columns if 'event_' in col or 'period_' in col or '_lag' in col])
    
    cols_to_drop = []
    
    for col in raw_sentiment_cols:
        if col in df.columns:
            for lag in range(1, news_lag + 1):
                df_lagged[f"{col}_lag{lag}"] = df[col].shift(lag)
            cols_to_drop.append(col)
    
    for col in sentiment_ma_cols:
        if col in df.columns and col not in cols_to_drop:
            if not any(pattern in col for pattern in no_lag_patterns):
                df_lagged[f"{col}_lag1"] = df[col].shift(1)
                cols_to_drop.append(col)
    
    for col in onchain_cols:
        if col not in exclude_cols:
            df_lagged[f"{col}_lag1"] = df[col].shift(onchain_lag)
            if col in df.columns:
                cols_to_drop.append(col)
    
    for col in other_cols:
        if col not in exclude_cols:
            df_lagged[f"{col}_lag1"] = df[col].shift(1)
            if col in df.columns:
                cols_to_drop.append(col)
    
    df_lagged.drop(columns=cols_to_drop, inplace=True, errors='ignore')
    return df_lagged


def add_price_lag_features_first(df):
    df_new = df.copy()
    close = df['ETH_Close']
    high = df['ETH_High']
    low = df['ETH_Low']
    volume = df['ETH_Volume']
    
    for lag in [1, 2, 3, 5, 7, 14, 21, 30]:
        df_new[f'close_lag{lag}'] = close.shift(lag)
    
    for lag in [1, 2, 3, 5, 7]:
        df_new[f'high_lag{lag}'] = high.shift(lag)
        df_new[f'low_lag{lag}'] = low.shift(lag)
        df_new[f'volume_lag{lag}'] = volume.shift(lag)
        df_new[f'return_lag{lag}'] = close.pct_change(periods=lag).shift(1)
    
    for lag in [1, 7, 30]:
        df_new[f'close_ratio_lag{lag}'] = close / close.shift(lag)
    
    return df_new

def add_interaction_features(df):
    df_interact = df.copy()
    
    if 'RSI_14' in df.columns and 'VOLUME_RATIO' in df.columns:
        df_interact['RSI_Volume_Strength'] = df['RSI_14'] * df['VOLUME_RATIO']
    
    if 'vix_VIX' in df.columns and 'VOLATILITY_20' in df.columns:
        df_interact['VIX_ETH_Vol_Cross'] = df['vix_VIX'] * df['VOLATILITY_20']
    
    if 'MACD_12_26_9' in df.columns and 'VOLUME_RATIO' in df.columns:
        df_interact['MACD_Volume_Momentum'] = df['MACD_12_26_9'] * df['VOLUME_RATIO']
    
    if 'btc_return' in df.columns and 'eth_btc_corr_30d' in df.columns:
        df_interact['BTC_Weighted_Impact'] = df['btc_return'] * df['eth_btc_corr_30d']
    
    if 'ATR_14' in df.columns and 'VOLUME_RATIO' in df.columns:
        df_interact['Liquidity_Risk'] = df['ATR_14'] * (1 / (df['VOLUME_RATIO'] + 1e-8))
    
    return df_interact

def add_volatility_regime_features(df):
    df_regime = df.copy()
    
    if 'VOLATILITY_20' in df.columns:
        vol_median = df['VOLATILITY_20'].rolling(60, min_periods=20).median()
        df_regime['vol_regime_high'] = (df['VOLATILITY_20'] > vol_median).astype(int)
        
        vol_mean = df['VOLATILITY_20'].rolling(30, min_periods=10).mean()
        vol_std = df['VOLATILITY_20'].rolling(30, min_periods=10).std()
        df_regime['vol_spike'] = (df['VOLATILITY_20'] > vol_mean + 2 * vol_std).astype(int)
        
        df_regime['vol_percentile_90d'] = df['VOLATILITY_20'].rolling(90, min_periods=30).apply(
            lambda x: (x.iloc[-1] > x).sum() / len(x) if len(x) > 0 else 0.5
        )
        df_regime['vol_trend'] = df['VOLATILITY_20'].pct_change(5)
        df_regime['vol_regime_duration'] = df_regime.groupby(
            (df_regime['vol_regime_high'] != df_regime['vol_regime_high'].shift()).cumsum()
        ).cumcount() + 1

    return df_regime


def add_normalized_price_lags(df):
    df_norm = df.copy()
    
    if 'ETH_Close' not in df.columns:
        return df_norm
    
    current_close = df['ETH_Close']
    lag_cols = [col for col in df.columns if 'close_lag' in col and col.replace('close_lag', '').isdigit()]
    
    for col in lag_cols:
        lag_num = col.replace('close_lag', '')
        df_norm[f'close_lag{lag_num}_ratio'] = df[col] / (current_close + 1e-8)
        next_lag_col = f'close_lag{int(lag_num)+1}'
        if next_lag_col in df.columns:
            df_norm[f'close_lag{lag_num}_logret'] = np.log(df[col] / (df[next_lag_col] + 1e-8))
    
    for col in df.columns:
        if 'high_lag' in col:
            lag_num = col.replace('high_lag', '')
            df_norm[f'high_lag{lag_num}_ratio'] = df[col] / (current_close + 1e-8)
        if 'low_lag' in col:
            lag_num = col.replace('low_lag', '')
            df_norm[f'low_lag{lag_num}_ratio'] = df[col] / (current_close + 1e-8)
    
    return df_norm


def add_percentile_features(df):
    df_pct = df.copy()
    
    if 'ETH_Close' in df.columns:
        df_pct['price_percentile_250d'] = df['ETH_Close'].rolling(250, min_periods=60).apply(
            lambda x: (x.iloc[-1] > x).sum() / len(x) if len(x) > 0 else 0.5
        )
    
    if 'ETH_Volume' in df.columns:
        df_pct['volume_percentile_90d'] = df['ETH_Volume'].rolling(90, min_periods=30).apply(
            lambda x: (x.iloc[-1] > x).sum() / len(x) if len(x) > 0 else 0.5
        )
    
    if 'RSI_14' in df.columns:
        df_pct['RSI_percentile_60d'] = df['RSI_14'].rolling(60, min_periods=20).apply(
            lambda x: (x.iloc[-1] > x).sum() / len(x) if len(x) > 0 else 0.5
        )
    
    return df_pct


def handle_missing_values_paper_based(df_clean, train_start_date, is_train=True, train_stats=None):
    """
    암호화폐 시계열 결측치 처리
    
    참고문헌:
    1. "Quantifying Cryptocurrency Unpredictability" (2025)

    2. "Time Series Data Forecasting" 
    
    3. "Dealing with Leaky Missing Data in Production" (2021)

    """
    
    # ===== 1. Lookback 제거 =====
    if isinstance(train_start_date, str):
        train_start_date = pd.to_datetime(train_start_date)
    
    before = len(df_clean)
    df_clean = df_clean[df_clean['date'] >= train_start_date].reset_index(drop=True)
    
    # ===== 2. Feature 컬럼 선택 =====
    target_cols = ['next_log_return', 'next_direction', 'next_close','next_open']
    feature_cols = [col for col in df_clean.columns 
                   if col not in target_cols + ['date']]
    
    # ===== 3. 결측 확인 =====
    missing_before = df_clean[feature_cols].isnull().sum().sum()
    
    # ===== 4. FFill → 0 =====
    df_clean[feature_cols] = df_clean[feature_cols].fillna(method='ffill')
    df_clean[feature_cols] = df_clean[feature_cols].fillna(0)
    
    missing_after = df_clean[feature_cols].isnull().sum().sum()
    
    # ===== 5. 무한대 처리 =====
    inf_count = 0
    for col in feature_cols:
        if np.isinf(df_clean[col]).sum() > 0:
            inf_count += np.isinf(df_clean[col]).sum()
            df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)
            df_clean[col] = df_clean[col].fillna(method='ffill').fillna(0)
    
    # ===== 6. 최종 확인 =====
    final_missing = df_clean[feature_cols].isnull().sum().sum()
    
    if final_missing > 0:
        df_clean[feature_cols] = df_clean[feature_cols].fillna(0)
    
    
    if is_train:
        return df_clean, {}
    else:
        return df_clean
    
    


def preprocess_non_stationary_features(df):
    df_proc = df.copy()
    
    prefixes_to_transform = [
        'eth_', 'aave_', 'lido_', 'makerdao_', 'uniswap_', 'curve_', 'chain_',
        'l2_', 'sp500_', 'gold_', 'dxy_', 'vix_', 'usdt_'
    ]
    
    exclude_prefixes = ['fg_', 'funding_']
    
    exclude_keywords = [
        '_pct_', '_ratio', '_lag', '_volatility', '_corr', '_beta', '_spread',
        'eth_return', 'btc_return', 'eth_log_return' 
    ]
    
    cols_to_transform = []
    for col in df_proc.columns:
        if col.startswith(tuple(prefixes_to_transform)):
            if not col.startswith(tuple(exclude_prefixes)):
                if not any(keyword in col for keyword in exclude_keywords):
                    cols_to_transform.append(col)
                    
    cols_to_drop = []

    for col in cols_to_transform:
        df_proc[col] = df_proc[col].fillna(method='ffill').replace(0, 1e-9)

        df_proc[f'{col}_pct_1d'] = df_proc[col].pct_change(1)
        df_proc[f'{col}_pct_5d'] = df_proc[col].pct_change(5)
        
        ma_30 = df_proc[col].rolling(window=30, min_periods=10).mean()
        df_proc[f'{col}_ma30_ratio'] = df_proc[col] / (ma_30 + 1e-9)
        
        cols_to_drop.append(col)

    df_proc = df_proc.drop(columns=cols_to_drop, errors='ignore')
    
    df_proc = df_proc.replace([np.inf, -np.inf], np.nan)
    df_proc = df_proc.fillna(method='ffill').fillna(0)
    
   
    return df_proc    
    

@jit(nopython=True)
def compute_triple_barrier_targets(
    prices_close,
    prices_high,
    prices_low,
    atr,
    lookahead_candles,
    atr_multiplier_profit,
    atr_multiplier_stop
):
    n = len(prices_close)
    targets_raw = np.zeros(n, dtype=np.int32)
    upper_barriers_arr = np.zeros(n, dtype=np.float64)
    lower_barriers_arr = np.zeros(n, dtype=np.float64)
    exit_reasons = np.zeros(n, dtype=np.int32)

    for i in range(n - lookahead_candles):
        current_atr = max(atr[i], 1e-8)
        current_price = prices_close[i]
        
        if np.isnan(current_atr) or current_price <= 0:
            continue
        
        upper_barrier = current_price + (current_atr * atr_multiplier_profit)
        lower_barrier = current_price - (current_atr * atr_multiplier_stop)
        
        upper_barriers_arr[i] = upper_barrier
        lower_barriers_arr[i] = lower_barrier

        profit_hit = False
        stop_hit = False
        
        for j in range(1, lookahead_candles + 1):
            future_high = prices_high[i + j]
            future_low = prices_low[i + j]
            
            if future_high >= upper_barrier:
                profit_hit = True
                targets_raw[i] = 1
                exit_reasons[i] = 1
                break
            
            if future_low <= lower_barrier:
                stop_hit = True
                targets_raw[i] = 2
                exit_reasons[i] = 2
                break
        
        if not profit_hit and not stop_hit:
            targets_raw[i] = 0
            exit_reasons[i] = 3

    return targets_raw, upper_barriers_arr, lower_barriers_arr, exit_reasons

def create_targets(df, lookahead_candles=3, atr_multiplier_profit=1.2, atr_multiplier_stop=0.8, trend_window=20, trend_analysis_points=None):
    
    df_target = df.copy()
    
    atr_col_name = 'ATR_14'
    if atr_col_name not in df.columns:
        raise ValueError(f"'{atr_col_name}' feature is missing. Run calculate_technical_indicators first.")

    sma_short = ta.sma(df_target['ETH_Close'], length=5)
    sma_long = ta.sma(df_target['ETH_Close'], length=trend_window)
    
    trend_directions = np.where(sma_short > sma_long, 1, -1).astype(int)
    trend_strengths = (df_target['ETH_Close'] - sma_long).abs() / (sma_long + 1e-8)
    trend_strengths = trend_strengths.fillna(0)

    df_target['trend_direction'] = trend_directions
    df_target['trend_strength'] = trend_strengths

    prices_close = df_target['ETH_Close'].to_numpy()
    prices_high = df_target['ETH_High'].to_numpy()
    prices_low = df_target['ETH_Low'].to_numpy()
    atr = pd.Series(df_target[atr_col_name]).fillna(method='ffill').fillna(0).to_numpy()

    targets_raw, upper_barriers, lower_barriers, exit_reasons = compute_triple_barrier_targets(
        prices_close, prices_high, prices_low, atr,
        lookahead_candles, atr_multiplier_profit, atr_multiplier_stop
    )

    next_open = df['ETH_Open'].shift(-1)
    next_close = df['ETH_Close'].shift(-1)

    df_target['next_open'] = next_open
    df_target['next_close'] = next_close
    df_target['next_log_return'] = np.log(next_close / next_open)

    df_target['next_direction'] = targets_raw
    df_target['take_profit_price'] = pd.Series(upper_barriers, index=df_target.index).replace(0, np.nan)
    df_target['stop_loss_price'] = pd.Series(lower_barriers, index=df_target.index).replace(0, np.nan)
    df_target['exit_reason'] = exit_reasons

    df_target['next_direction'].iloc[-lookahead_candles:] = np.nan

    return df_target

In [4]:
# ==========================================================================
# 특징 선택 함수 (Feature Selection)
# ==========================================================================

def select_features_verified(X_train, y_train, task='class', top_n=30, verbose=True):
    
    if task == 'class':
        # 상호 정보량 (Mutual Information)
        mi_scores = mutual_info_classif(X_train, y_train, random_state=42, n_neighbors=3)
    else:
        mi_scores = mutual_info_regression(X_train, y_train, random_state=42, n_neighbors=3)
    
    mi_idx = np.argsort(mi_scores)[::-1][:top_n]
    mi_features = X_train.columns[mi_idx].tolist()
    
    # RFE Estimator (LGBM)
    if task == 'class':
        estimator = LGBMClassifier(
            n_estimators=100,
            learning_rate=0.05,
            max_depth=5,
            random_state=42,
            verbose=-1
        )
    else:
        estimator = LGBMRegressor(
            n_estimators=100,
            learning_rate=0.05,
            max_depth=5,
            random_state=42,
            verbose=-1
        )
    
    rfe = RFE(
        estimator=estimator,
        n_features_to_select=top_n,
        step=0.1,
        verbose=0
    )
    
    rfe.fit(X_train, y_train)
    rfe_features = X_train.columns[rfe.support_].tolist()

    # Random Forest Importance
    if task == 'class':
        rf_model = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            class_weight='balanced', # 클래스 불균형 고려
            random_state=42,
            n_jobs=-1
        )
    else:
        rf_model = RandomForestRegressor(
            n_estimators=100,
            max_depth=10,
            random_state=42,
            n_jobs=-1
        )
    
    rf_model.fit(X_train, y_train)
    rf_importances = rf_model.feature_importances_
    rf_idx = np.argsort(rf_importances)[::-1][:top_n]
    rf_features = X_train.columns[rf_idx].tolist()
    
    # 앙상블 투표
    all_features = mi_features + rfe_features + rf_features
    feature_votes = Counter(all_features)
    selected_features = [feat for feat, _ in feature_votes.most_common(top_n)]

    if len(selected_features) < top_n:
        remaining = top_n - len(selected_features)
        for feat in mi_features:
            if feat not in selected_features:
                selected_features.append(feat)
                remaining -= 1
                if remaining == 0:
                    break
    
    return selected_features, {
        'mi_features': mi_features,
        'rfe_features': rfe_features,
        'rf_features': rf_features,
        'feature_votes': feature_votes,
        'mi_scores': dict(zip(X_train.columns, mi_scores)),
        'rf_importances': dict(zip(X_train.columns, rf_importances))
    }


def select_features_multi_target(X_train, y_train, target_type='direction', top_n=30):
    
    atr_col_name = 'ATR_14'

    if target_type == 'direction':
        selected, stats = select_features_verified(
            X_train,  
            y_train['next_direction'],  
            task='class',  
            top_n=top_n
        )
        
        # ATR_14가 제외되었을 경우 필수적으로 포함 (안정성 확보)
        if atr_col_name not in selected and atr_col_name in X_train.columns:
            if len(selected) > 0:
                 # 가장 덜 중요한 요소를 제거하고 ATR_14를 추가
                selected.pop() 
            selected.insert(0, atr_col_name)
    
    
    print(", ".join(selected))
    return selected, stats

# ==========================================================================
# 데이터 분할 및 처리 함수
# ==========================================================================

def split_walk_forward_method(df, train_start_date,
                             final_test_start='2025-01-01',
                             initial_train_size=800,
                             val_size=150,
                             test_size=150,
                             step=150,
                             gap_size=20,
                             apply_gap_to_train_val=True):
    
    df_full_period = df[df['date'] >= train_start_date].copy()
    df_full_period = df_full_period.sort_values('date').reset_index(drop=True)
    
    if isinstance(final_test_start, str):
        final_test_start = pd.to_datetime(final_test_start)
        
    final_test_df = df_full_period[df_full_period['date'] >= final_test_start].copy()
    
    rolling_cutoff_date = final_test_start - pd.Timedelta(days=gap_size)
    df_rolling_period = df_full_period[df_full_period['date'] < rolling_cutoff_date].copy().reset_index(drop=True)

    total_rolling_days = len(df_rolling_period)
    
    if apply_gap_to_train_val:
        min_required_days = initial_train_size + gap_size + val_size + gap_size + test_size
    else:
        min_required_days = initial_train_size + val_size + gap_size + test_size
    
    if total_rolling_days < min_required_days:
        n_splits = 0
    else:
        n_splits = (total_rolling_days - min_required_days) // step + 1
    
    folds = []
    
    for fold_idx in range(n_splits):
        test_end_idx = total_rolling_days - (fold_idx * step)
        test_start_idx = test_end_idx - test_size
        
        if test_start_idx < 0: break
        
        val_end_idx = test_start_idx - gap_size
        val_start_idx = val_end_idx - val_size
        
        if apply_gap_to_train_val:
            train_end_idx = val_start_idx - gap_size
        else:
            train_end_idx = val_start_idx
            
        train_start_idx = train_end_idx - initial_train_size
        
        if train_start_idx < 0: break
        
        train_fold = df_rolling_period.iloc[train_start_idx:train_end_idx].copy()
        val_fold = df_rolling_period.iloc[val_start_idx:val_end_idx].copy()
        test_fold = df_rolling_period.iloc[test_start_idx:test_end_idx].copy()
        
        folds.append({
            'train': train_fold, 'val': val_fold, 'test': test_fold,
            'fold_idx': fold_idx + 1, 'fold_type': 'walk_forward_rolling'
        })
        
    folds.reverse()
    for idx, fold in enumerate(folds):
        fold['fold_idx'] = idx + 1

    if len(final_test_df) > 0 and len(df_rolling_period) > 0:
        
        final_val_end_idx = len(df_rolling_period) 
        final_val_start_idx = final_val_end_idx - val_size
        
        if apply_gap_to_train_val:
            final_train_end_idx = final_val_start_idx - gap_size
        else:
            final_train_end_idx = final_val_start_idx
            
        final_train_start_idx = max(0, final_train_end_idx - initial_train_size)
        
        final_train_data = df_rolling_period.iloc[final_train_start_idx:final_train_end_idx].copy()
        final_val_data = df_rolling_period.iloc[final_val_start_idx:final_val_end_idx].copy()
        
        folds.append({
            'train': final_train_data,
            'val': final_val_data,
            'test': final_test_df,
            'fold_idx': len(folds) + 1,
            'fold_type': 'final_holdout'
        })
    
    return folds

def process_single_split(split_data, target_type='direction', top_n=40, fold_idx=None, trend_params=None):
    train_df = split_data['train']
    val_df = split_data['val']
    test_df = split_data['test']
    fold_type = split_data.get('fold_type', 'unknown')
    
    train_processed, missing_stats = handle_missing_values_paper_based(
        train_df.copy(), train_start_date=train_df['date'].min(), is_train=True
    )
    
    val_processed = handle_missing_values_paper_based(
        val_df.copy(), train_start_date=val_df['date'].min(),
        is_train=False, train_stats=missing_stats
    )
    
    test_processed = handle_missing_values_paper_based(
        test_df.copy(), train_start_date=test_df['date'].min(),
        is_train=False, train_stats=missing_stats
    )
    
    target_cols = ['next_direction','next_log_return', 'next_close','next_open', 
                   'take_profit_price', 'stop_loss_price',
                   'trend_direction', 'trend_strength', 'exit_reason']
    
    dropna_cols = ['next_direction','next_log_return', 'next_close','next_open', 
                   'take_profit_price', 'stop_loss_price']

    train_processed = train_processed.dropna(subset=dropna_cols).reset_index(drop=True)
    val_processed = val_processed.dropna(subset=dropna_cols).reset_index(drop=True)
    test_processed = test_processed.dropna(subset=dropna_cols).reset_index(drop=True)

    feature_cols = [col for col in train_processed.columns 
                    if col not in target_cols + ['date']]
    
    X_train = train_processed[feature_cols]
    y_train = train_processed[target_cols]
    
    X_val = val_processed[feature_cols]
    y_val = val_processed[target_cols]
    
    X_test = test_processed[feature_cols]
    y_test = test_processed[target_cols]

    # ==========================================================================
    # 클래스 불균형 분석 코드 추가
    # ==========================================================================
    def analyze_class_balance(y_df, name):
        target_col = 'next_direction'
        if target_col not in y_df.columns:
            return ""
        
        counts = y_df[target_col].value_counts(normalize=True).sort_index()
        total_n = len(y_df)
        
        # 클래스 0, 1, 2의 비율을 포맷팅
        ratio_0 = counts.get(0, 0)
        ratio_1 = counts.get(1, 0)
        ratio_2 = counts.get(2, 0)

        # 타겟 클래스 맵핑: 0:Hold, 1:Long, 2:Short
        output = f" {name} Set (N={total_n}): Hold(0): {ratio_0:.2%} | Long(1): {ratio_1:.2%} | Short(2): {ratio_2:.2%}"
        
        return output

    print(f"\n--- Fold {fold_idx} ({fold_type}) Class Balance Analysis (top_n={top_n}) ---")
    print(analyze_class_balance(y_train, 'Train'))
    print(analyze_class_balance(y_val, 'Validation'))
    print(analyze_class_balance(y_test, 'Test'))
    print("-" * 60)
    # ==========================================================================


    selected_features, selection_stats = select_features_multi_target(
        X_train, y_train, target_type=target_type, top_n=top_n
    )
    
    X_train_sel = X_train[selected_features]
    X_val_sel = X_val[selected_features]
    X_test_sel = X_test[selected_features]
    
    robust_scaler = RobustScaler()
    standard_scaler = StandardScaler()
    
    X_train_robust = robust_scaler.fit_transform(X_train_sel)
    X_val_robust = robust_scaler.transform(X_val_sel)
    X_test_robust = robust_scaler.transform(X_test_sel)
    
    X_train_standard = standard_scaler.fit_transform(X_train_sel)
    X_val_standard = standard_scaler.transform(X_val_sel)
    X_test_standard = standard_scaler.transform(X_test_sel)
    
    if trend_params is None:
        trend_params = {}
        
    result = {
        'train': {
            'X_robust': X_train_robust,
            'X_standard': X_train_standard,
            'X_raw': X_train_sel,
            'y': y_train.reset_index(drop=True), 
            'dates': train_processed['date'].reset_index(drop=True) 
        },
        'val': {
            'X_robust': X_val_robust,
            'X_standard': X_val_standard,
            'X_raw': X_val_sel,
            'y': y_val.reset_index(drop=True), 
            'dates': val_processed['date'].reset_index(drop=True)
        },
        'test': {
            'X_robust': X_test_robust,
            'X_standard': X_test_standard,
            'X_raw': X_test_sel,
            'y': y_test.reset_index(drop=True),
            'dates': test_processed['date'].reset_index(drop=True)
        },
        'scaler': robust_scaler, 
        'stats': {
            'robust_scaler': robust_scaler,
            'standard_scaler': standard_scaler,
            'selected_features': selected_features,
            'selection_stats': selection_stats,
            'missing_stats': missing_stats,  
            'target_type': target_type,
            'target_cols': target_cols,
            'fold_type': fold_type,
            'fold_idx': fold_idx,
            'trend_window': trend_params.get('trend_window', 120),
            'trend_analysis_points': trend_params.get('trend_analysis_points', 5)
        }
    }
    
    return result

def build_complete_pipeline_corrected(df_raw, train_start_date, 
                                      final_test_start='2025-01-01',
                                      method='tvt', target_type='direction',
                                      lookahead_candles=8, atr_multiplier_profit=1.5, 
                                      atr_multiplier_stop=1.0, **kwargs):
    df = df_raw.copy()
    
    df = add_price_lag_features_first(df)
    df = calculate_technical_indicators(df)
    df = add_enhanced_cross_crypto_features(df)
    df = add_volatility_regime_features(df)
    df = add_interaction_features(df)
    df = add_percentile_features(df)
    df = add_normalized_price_lags(df)
    df = preprocess_non_stationary_features(df)
    df = apply_lag_features(df, news_lag=2, onchain_lag=1)
    
    trend_params = {
        'trend_window': kwargs.get('trend_window', 120),
        'trend_analysis_points': kwargs.get('trend_analysis_points', 5)
    }
    
    df = create_targets(
        df, 
        lookahead_candles=lookahead_candles,
        atr_multiplier_profit=atr_multiplier_profit, 
        atr_multiplier_stop=atr_multiplier_stop,
        **trend_params  
    )
    
    df = remove_raw_prices_and_transform(df, target_type, method)
    df = df.iloc[:-lookahead_candles]
    
    split_kwargs = {
        'final_test_start': final_test_start,
        'gap_size': lookahead_candles, 
        'apply_gap_to_train_val': True
    }
    
    for key in ['initial_train_size', 'val_size', 'test_size', 'step']:
        if key in kwargs:
            split_kwargs[key] = kwargs[key]
    
    splits = split_walk_forward_method(df, train_start_date, **split_kwargs)

    # top_n을 20으로 설정하여 특징 개수 축소 (기존 30/40에서 조정)
    top_n_features = kwargs.get('top_n', 20) 
    
    result = [
        process_single_split(
            fold, 
            target_type=target_type,  
            top_n=top_n_features,
            fold_idx=fold['fold_idx'],
            trend_params=trend_params
        ) 
        for fold in splits
    ]
    
    summary_data = []
    for fold in result:
        f_idx = fold['stats']['fold_idx']
        f_type = fold['stats']['fold_type']
        tr_dates = fold['train']['dates']
        val_dates = fold['val']['dates']
        te_dates = fold['test']['dates']
        
        summary_data.append({
            'Fold': f_idx,
            'Type': f_type,
            'Train_N': len(fold['train']['y']),
            'Train_Start': tr_dates.min().date() if len(tr_dates)>0 else '-',
            'Train_End': tr_dates.max().date() if len(tr_dates)>0 else '-',
            'Val_N': len(fold['val']['y']),
            'Val_Start': val_dates.min().date() if len(val_dates)>0 else '-',
            'Val_End': val_dates.max().date() if len(val_dates)>0 else '-',
            'Test_N': len(fold['test']['y']),
            'Test_Start': te_dates.min().date() if len(te_dates)>0 else '-',
            'Test_End': te_dates.max().date() if len(te_dates)>0 else '-'
        })
    
    summary_df = pd.DataFrame(summary_data)
    
    print("\n" + "="*100)
    print(" [FINAL PIPELINE SUMMARY] dropna() 이후 데이터 생존 현황")
    print("="*100)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    print(summary_df.to_string(index=False))
    print("="*100 + "\n")
    
    return result

In [5]:
class DirectionModels:
    
    # ==========================================================================
    # 1. Random Forest (ML)
    # ==========================================================================
    @staticmethod
    def random_forest(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            param = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 300),
                'max_depth': trial.suggest_int('max_depth', 4, 10),
                'min_samples_split': trial.suggest_int('min_samples_split', 10, 40),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 4, 20),
                'max_features': 'sqrt',
                'random_state': 42,
                'n_jobs': -1,
                'class_weight': 'balanced'
            }
            
            model = RandomForestClassifier(**param)
            model.fit(X_train, y_train)
            
            val_pred_proba = model.predict_proba(X_val)
            return log_loss(y_val, val_pred_proba)
        
        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=15)
        
        best_model = RandomForestClassifier(**study.best_params, class_weight='balanced', random_state=42, n_jobs=-1)
        best_model.fit(X_train, y_train)
        
        # 과적합 확인용 출력
        train_acc = accuracy_score(y_train, best_model.predict(X_train))
        val_acc = accuracy_score(y_val, best_model.predict(X_val))
        print(f"[RandomForest] Acc: {train_acc:.4f}/{val_acc:.4f} (Gap: {train_acc-val_acc:.4f})")
        
        return best_model

    # ==========================================================================
    # 2. LightGBM (ML)
    # ==========================================================================
    @staticmethod
    def lightgbm(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 300),
                'max_depth': trial.suggest_int('max_depth', 3, 8),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 20, 60),
                'objective': 'multiclass',
                'num_class': 3,
                'metric': 'multi_logloss',
                'class_weight': 'balanced',
                'random_state': 42,
                'verbose': -1,
                'force_col_wise': True
            }

            model = LGBMClassifier(**params)
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[early_stopping(20, verbose=False)])
            return model.best_score_['valid_0']['multi_logloss']

        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=20)

        best_params = study.best_params
        best_params.update({'objective': 'multiclass', 'num_class': 3, 'metric': 'multi_logloss', 'class_weight': 'balanced', 'random_state': 42, 'verbose': -1, 'force_col_wise': True})
        
        final_model = LGBMClassifier(**best_params)
        final_model.fit(X_train, y_train)
        
        train_acc = accuracy_score(y_train, final_model.predict(X_train))
        val_acc = accuracy_score(y_val, final_model.predict(X_val))
        print(f"[LightGBM] Acc: {train_acc:.4f}/{val_acc:.4f} (Gap: {train_acc-val_acc:.4f})")
        
        return final_model

    # ==========================================================================
    # 3. XGBoost (ML)
    # ==========================================================================
    @staticmethod
    def xgboost(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        sample_weights = compute_sample_weight('balanced', y_train)

        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 300),
                'max_depth': trial.suggest_int('max_depth', 3, 8),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
                'subsample': trial.suggest_float('subsample', 0.5, 0.8),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.8),
                'objective': 'multi:softprob',
                'num_class': 3,
                'eval_metric': 'mlogloss',
                'tree_method': 'hist',
                'random_state': 42,
                'n_jobs': -1
            }

            model = XGBClassifier(**params)
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], sample_weight=sample_weights, verbose=False)
            return model.evals_result()['validation_0']['mlogloss'][-1]

        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=20)

        best_params = study.best_params
        best_params.update({'objective': 'multi:softprob', 'num_class': 3, 'eval_metric': 'mlogloss', 'tree_method': 'hist', 'random_state': 42, 'n_jobs': -1})
        
        final_model = XGBClassifier(**best_params)
        final_model.fit(X_train, y_train, sample_weight=sample_weights)
        
        train_acc = accuracy_score(y_train, final_model.predict(X_train))
        val_acc = accuracy_score(y_val, final_model.predict(X_val))
        print(f"[XGBoost] Acc: {train_acc:.4f}/{val_acc:.4f} (Gap: {train_acc-val_acc:.4f})")
        
        return final_model

    # ==========================================================================
    # 5. Logistic Regression (ML)
    # ==========================================================================
    @staticmethod
    def logistic_regression(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            param = {
                'C': trial.suggest_float('C', 0.01, 10.0, log=True),
                'penalty': 'l2',
                'solver': 'lbfgs',
                'multi_class': 'multinomial',
                'class_weight': 'balanced',
                'max_iter': 2000,
                'random_state': 42,
                'n_jobs': -1
            }
            
            model = LogisticRegression(**param)
            model.fit(X_train, y_train)
            val_pred_proba = model.predict_proba(X_val)
            return log_loss(y_val, val_pred_proba)

        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=15)
        
        best_model = LogisticRegression(**study.best_params, penalty='l2', solver='lbfgs', multi_class='multinomial', class_weight='balanced', max_iter=2000, random_state=42, n_jobs=-1)
        best_model.fit(X_train, y_train)
        
        train_acc = accuracy_score(y_train, best_model.predict(X_train))
        val_acc = accuracy_score(y_val, best_model.predict(X_val))
        print(f"[LogisticReg] Acc: {train_acc:.4f}/{val_acc:.4f} (Gap: {train_acc-val_acc:.4f})")
        
        return best_model

    # ==========================================================================
    # 9. CatBoost (ML)
    # ==========================================================================
    @staticmethod
    def catboost(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        y_train = y_train.astype(int)
        y_val = y_val.astype(int)

        def objective(trial):
            params = {
                'iterations': trial.suggest_int('iterations', 100, 300),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
                'depth': trial.suggest_int('depth', 3, 8),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
                'loss_function': 'MultiClass',
                'eval_metric': 'MultiClass',
                'auto_class_weights': 'Balanced',
                'random_seed': 42,
                'verbose': False,
                'early_stopping_rounds': 20,
                'allow_writing_files': False
            }
            
            model = CatBoostClassifier(**params)
            model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)
            
            val_pred_proba = model.predict_proba(X_val)
            
            
            if val_pred_proba.shape[1] < 3:
                padded = np.zeros((val_pred_proba.shape[0], 3))

                padded[:, :val_pred_proba.shape[1]] = val_pred_proba
                val_pred_proba = padded

            try:
                return log_loss(y_val, val_pred_proba, labels=[0, 1, 2])
            except ValueError:

                acc = accuracy_score(y_val, model.predict(X_val))
                return 1.0 - acc

        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=20)

        best_params = study.best_params
        best_params.update({
            'loss_function': 'MultiClass', 
            'eval_metric': 'MultiClass', 
            'auto_class_weights': 'Balanced', 
            'random_seed': 42, 
            'verbose': False, 
            'allow_writing_files': False
        })
        
        final_model = CatBoostClassifier(**best_params)
        final_model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)

        train_pred = final_model.predict(X_train).ravel()
        val_pred = final_model.predict(X_val).ravel()
        
        train_acc = accuracy_score(y_train, train_pred)
        val_acc = accuracy_score(y_val, val_pred)
        
        print(f"[CatBoost] Acc: {train_acc:.4f}/{val_acc:.4f} (Gap: {train_acc-val_acc:.4f})")
        
        return final_model

    # ==========================================================================
    # 13. Gradient Boosting (ML)
    # ==========================================================================
    @staticmethod
    def gradient_boosting(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        sample_weights = compute_sample_weight('balanced', y_train)
        
        def objective(trial):
            param = {
                'n_estimators': trial.suggest_int('n_estimators', 80, 200),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
                'max_depth': trial.suggest_int('max_depth', 2, 6),
                'subsample': trial.suggest_float('subsample', 0.5, 0.9),
                'validation_fraction': 0.1,
                'n_iter_no_change': 10,
                'random_state': 42
            }
            
            model = GradientBoostingClassifier(**param)
            model.fit(X_train, y_train, sample_weight=sample_weights)
            val_pred_proba = model.predict_proba(X_val)
            return log_loss(y_val, val_pred_proba)

        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=15)
        
        best_model = GradientBoostingClassifier(**study.best_params, random_state=42)
        best_model.fit(X_train, y_train, sample_weight=sample_weights)
        
        train_acc = accuracy_score(y_train, best_model.predict(X_train))
        val_acc = accuracy_score(y_val, best_model.predict(X_val))
        print(f"[GradientBoost] Acc: {train_acc:.4f}/{val_acc:.4f} (Gap: {train_acc-val_acc:.4f})")
        
        return best_model

    # ==========================================================================
    # 14. HistGradientBoosting (ML)
    # ==========================================================================
    @staticmethod
    def histgradient_boosting(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        sample_weights = compute_sample_weight('balanced', y_train)

        def objective(trial):
            params = {
                'max_iter': trial.suggest_int('max_iter', 100, 300),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
                'max_depth': trial.suggest_int('max_depth', 2, 8),
                'l2_regularization': trial.suggest_float('l2_regularization', 0.1, 10.0, log=True),
                'early_stopping': True,
                'validation_fraction': 0.1,
                'n_iter_no_change': 10,
                'random_state': 42
            }
            
            model = HistGradientBoostingClassifier(**params)
            model.fit(X_train, y_train, sample_weight=sample_weights)
            val_pred_proba = model.predict_proba(X_val)
            return log_loss(y_val, val_pred_proba)

        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=15)
        
        best_model = HistGradientBoostingClassifier(**study.best_params, early_stopping=True, random_state=42)
        best_model.fit(X_train, y_train, sample_weight=sample_weights)
        
        train_acc = accuracy_score(y_train, best_model.predict(X_train))
        val_acc = accuracy_score(y_val, best_model.predict(X_val))
        print(f"[HistGradient] Acc: {train_acc:.4f}/{val_acc:.4f} (Gap: {train_acc-val_acc:.4f})")
        
        return best_model

    # ==========================================================================
    # 15. Stacking Ensemble (ML)
    # ==========================================================================
    @staticmethod
    def stacking_ensemble(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        base_learners = [
            ('xgb', XGBClassifier(n_estimators=100, max_depth=3, objective='multi:softprob', num_class=3, eval_metric='mlogloss', n_jobs=-1, random_state=42)),
            ('lgbm', LGBMClassifier(n_estimators=100, max_depth=3, objective='multiclass', num_class=3, class_weight='balanced', n_jobs=-1, random_state=42, verbose=-1)),
            ('hist', HistGradientBoostingClassifier(max_iter=100, max_depth=3, random_state=42))
        ]
        
        meta_learner = LogisticRegression(multi_class='multinomial', solver='lbfgs', class_weight='balanced', max_iter=1000, random_state=42)
        
        model = StackingClassifier(
            estimators=base_learners,
            final_estimator=meta_learner,
            cv=3,
            n_jobs=-1,
            passthrough=False
        )
        
        model.fit(X_train, y_train)
        
        train_acc = accuracy_score(y_train, model.predict(X_train))
        val_acc = accuracy_score(y_val, model.predict(X_val))
        print(f"[Stacking] Acc: {train_acc:.4f}/{val_acc:.4f} (Gap: {train_acc-val_acc:.4f})")
        
        return model

    # ==========================================================================
    # 19. LSTM (DL)
    # ==========================================================================
    @staticmethod
    def lstm(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        class_weights_array = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        class_weight_dict = {i: w for i, w in enumerate(class_weights_array)}

        def objective(trial):
            units = trial.suggest_int('units', 32, 128, step=32)
            dropout = trial.suggest_float('dropout', 0.2, 0.5)
            lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
            
            model = Sequential([
                LSTM(units, input_shape=input_shape, return_sequences=False),
                Dropout(dropout),
                Dense(32, activation='relu'),
                Dropout(dropout),
                Dense(3, activation='softmax') # 3-Class
            ])
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=15, batch_size=64, class_weight=class_weight_dict, verbose=0)
            return history.history['val_loss'][-1]

        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=10)
        
        best_params = study.best_params
        model = Sequential([
            LSTM(best_params['units'], input_shape=input_shape, return_sequences=False),
            Dropout(best_params['dropout']),
            Dense(32, activation='relu'),
            Dropout(best_params['dropout']),
            Dense(3, activation='softmax')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['lr']), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        # 재학습
        model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=64, class_weight=class_weight_dict, verbose=0)
        
        _, train_acc = model.evaluate(X_train, y_train, verbose=0)
        _, val_acc = model.evaluate(X_val, y_val, verbose=0)
        print(f"[LSTM] Acc: {train_acc:.4f}/{val_acc:.4f} (Gap: {train_acc-val_acc:.4f})")
        
        return model

    # ==========================================================================
    # 20. BiLSTM (DL)
    # ==========================================================================
    @staticmethod
    def bilstm(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        class_weights_array = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        class_weight_dict = {i: w for i, w in enumerate(class_weights_array)}

        def objective(trial):
            units = trial.suggest_int('units', 32, 128, step=32)
            dropout = trial.suggest_float('dropout', 0.2, 0.5)
            lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
            
            model = Sequential([
                Bidirectional(LSTM(units, return_sequences=False), input_shape=input_shape),
                Dropout(dropout),
                Dense(32, activation='relu'),
                Dropout(dropout),
                Dense(3, activation='softmax')
            ])
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=15, batch_size=64, class_weight=class_weight_dict, verbose=0)
            return history.history['val_loss'][-1]

        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=10)
        
        best_params = study.best_params
        model = Sequential([
            Bidirectional(LSTM(best_params['units'], return_sequences=False), input_shape=input_shape),
            Dropout(best_params['dropout']),
            Dense(32, activation='relu'),
            Dropout(best_params['dropout']),
            Dense(3, activation='softmax')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['lr']), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=64, class_weight=class_weight_dict, verbose=0)
        
        _, train_acc = model.evaluate(X_train, y_train, verbose=0)
        _, val_acc = model.evaluate(X_val, y_val, verbose=0)
        print(f"[BiLSTM] Acc: {train_acc:.4f}/{val_acc:.4f} (Gap: {train_acc-val_acc:.4f})")
        
        return model

    # ==========================================================================
    # 21. GRU (DL)
    # ==========================================================================
    @staticmethod
    def gru(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        class_weights_array = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        class_weight_dict = {i: w for i, w in enumerate(class_weights_array)}

        def objective(trial):
            units = trial.suggest_int('units', 32, 128, step=32)
            dropout = trial.suggest_float('dropout', 0.2, 0.5)
            lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
            
            model = Sequential([
                GRU(units, input_shape=input_shape, return_sequences=False),
                Dropout(dropout),
                Dense(32, activation='relu'),
                Dropout(dropout),
                Dense(3, activation='softmax')
            ])
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=15, batch_size=64, class_weight=class_weight_dict, verbose=0)
            return history.history['val_loss'][-1]

        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=10)
        
        best_params = study.best_params
        model = Sequential([
            GRU(best_params['units'], input_shape=input_shape, return_sequences=False),
            Dropout(best_params['dropout']),
            Dense(32, activation='relu'),
            Dropout(best_params['dropout']),
            Dense(3, activation='softmax')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['lr']), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=64, class_weight=class_weight_dict, verbose=0)
        
        _, train_acc = model.evaluate(X_train, y_train, verbose=0)
        _, val_acc = model.evaluate(X_val, y_val, verbose=0)
        print(f"[GRU] Acc: {train_acc:.4f}/{val_acc:.4f} (Gap: {train_acc-val_acc:.4f})")
        
        return model

In [6]:
# ML Models (8개) 
ML_MODELS_CLASSIFICATION = [
    {'index': 1, 'name': 'CatBoost', 'func': DirectionModels.catboost, 'needs_val': True},
    {'index': 2, 'name': 'RandomForest', 'func': DirectionModels.random_forest, 'needs_val': True},
    {'index': 3, 'name': 'LightGBM', 'func': DirectionModels.lightgbm, 'needs_val': True},
    {'index': 4, 'name': 'XGBoost', 'func': DirectionModels.xgboost, 'needs_val': True},
    {'index': 5, 'name': 'GradientBoosting', 'func': DirectionModels.gradient_boosting, 'needs_val': True},
    {'index': 6, 'name': 'HistGradientBoosting', 'func': DirectionModels.histgradient_boosting, 'needs_val': True},
    {'index': 7, 'name': 'LogisticRegression', 'func': DirectionModels.logistic_regression, 'needs_val': True},
    {'index': 8, 'name': 'StackingEnsemble', 'func': DirectionModels.stacking_ensemble, 'needs_val': True},
]

# DL Models (3개)
DL_MODELS_CLASSIFICATION = [
    {'index': 9, 'name': 'LSTM', 'func': DirectionModels.lstm, 'needs_val': True},
    {'index': 10, 'name': 'BiLSTM', 'func': DirectionModels.bilstm, 'needs_val': True},
    {'index': 11, 'name': 'GRU', 'func': DirectionModels.gru, 'needs_val': True}
]

In [7]:
import pandas as pd
import numpy as np
import os
import gc
import tensorflow as tf
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report

class ModelEvaluator:
    def __init__(self, save_models=False):
        self.results = []
        self.predictions = {}
        self.models = {} if save_models else None
        self.save_models = save_models
    
    def _predict_model(self, model, X):
        pred = model.predict(X)
        if isinstance(pred, list):
            cleaned = []
            for p in pred:
                if isinstance(p, np.ndarray):
                    cleaned.append(p.squeeze() if p.shape[-1] == 1 else p)
                else:
                    cleaned.append(p)
            return np.array(cleaned)
        else:
            return pred.squeeze() if pred.shape[-1] == 1 else pred

    def _ensure_3class_proba(self, proba):
        if proba is None:
            return None
        
        if isinstance(proba, list):
            proba = np.array(proba)
            
        if proba.ndim == 1:
            proba = proba.reshape(-1, 1)

        n_samples, n_classes = proba.shape
        
        if n_classes >= 3:
            return proba
        
        padded = np.zeros((n_samples, 3))
        padded[:, :n_classes] = proba
        return padded

    def evaluate_classification_model(self, model, X_train, y_train, X_val, y_val, 
                                    X_test, y_test_df, test_dates, model_name,
                                    is_deep_learning=False):
        
        if is_deep_learning:
            train_pred_proba = self._predict_model(model, X_train)
            val_pred_proba = self._predict_model(model, X_val)
            test_pred_proba = self._predict_model(model, X_test)
            
            if isinstance(test_pred_proba, list): test_pred_proba = test_pred_proba[0]
            if isinstance(train_pred_proba, list): train_pred_proba = train_pred_proba[0]
            if isinstance(val_pred_proba, list): val_pred_proba = val_pred_proba[0]

        else:
            train_pred = self._predict_model(model, X_train)
            val_pred = self._predict_model(model, X_val)
            test_pred = self._predict_model(model, X_test)
            
            if hasattr(model, 'predict_proba'):
                train_pred_proba = model.predict_proba(X_train)
                val_pred_proba = model.predict_proba(X_val)
                test_pred_proba = model.predict_proba(X_test)
            else:
                n_classes = 3
                train_pred_proba = np.eye(n_classes)[train_pred.astype(int)]
                val_pred_proba = np.eye(n_classes)[val_pred.astype(int)]
                test_pred_proba = np.eye(n_classes)[test_pred.astype(int)]

        train_pred_proba = self._ensure_3class_proba(train_pred_proba)
        val_pred_proba = self._ensure_3class_proba(val_pred_proba)
        test_pred_proba = self._ensure_3class_proba(test_pred_proba)

        train_pred = np.argmax(train_pred_proba, axis=1)
        val_pred = np.argmax(val_pred_proba, axis=1)
        test_pred = np.argmax(test_pred_proba, axis=1)
        
        y_test_direction = y_test_df['next_direction'].values.astype(int)
        y_train = y_train.astype(int)
        y_val = y_val.astype(int)

        # 1. 기본 지표 계산
        train_acc = accuracy_score(y_train, train_pred)
        val_acc = accuracy_score(y_val, val_pred)
        test_acc = accuracy_score(y_test_direction, test_pred)
        

        report = classification_report(y_test_direction, test_pred, output_dict=True, zero_division=0, labels=[0, 1, 2])
        
        # 0: 관망, 1: 롱, 2: 숏
        prec_neutral = report['0']['precision']
        prec_long = report['1']['precision']   # 여기가 진짜 롱 승률
        prec_short = report['2']['precision']  # 여기가 진짜 숏 승률
        
        rec_long = report['1']['recall']
        rec_short = report['2']['recall']
        
        f1_long = report['1']['f1-score']
        
        # Weighted Average (참고용으로 유지)
        test_prec_weighted = precision_score(y_test_direction, test_pred, average='weighted', zero_division=0)
        
        try:
            test_roc_auc = roc_auc_score(y_test_direction, test_pred_proba, multi_class='ovr')
        except:
            test_roc_auc = 0.0

        self._save_predictions(model_name, test_pred, test_pred_proba, y_test_df, test_dates)
        
        if self.save_models and self.models is not None:
            self.models[model_name] = model
        
        self.results.append({
            'Model': model_name, 
            'Train_Accuracy': train_acc, 
            'Val_Accuracy': val_acc,
            'Test_Accuracy': test_acc, 
            
            'Precision_Long': prec_long,
            'Precision_Short': prec_short,
            'Precision_Neutral': prec_neutral,
            
            'Recall_Long': rec_long,
            'Recall_Short': rec_short,
            'F1_Long': f1_long,
            
            # 전체 지표 (참고용)
            'Test_Precision_Weighted': test_prec_weighted, 
            'Test_AUC_ROC': test_roc_auc
        })
        
        # 4. 터미널 출력도 상세하게 변경
        print(f"[{model_name}] Total Acc: {test_acc:.4f} | Long(1) Prec: {prec_long:.4f} | Short(2) Prec: {prec_short:.4f}")
        
        gc.collect()
        return self.results[-1]
    
    def _save_predictions(self, model_name, pred_direction, pred_proba, y_test_df, dates):
        pred_proba = self._ensure_3class_proba(pred_proba)

        prob_neutral = pred_proba[:, 0]
        prob_long = pred_proba[:, 1]
        prob_short = pred_proba[:, 2]
        
        trade_confidence = np.where(pred_direction == 1, prob_long, 
                                  np.where(pred_direction == 2, prob_short, 0.0))

        y_test_direction = y_test_df['next_direction'].values.astype(int)

        predictions_df = pd.DataFrame({
            'date': dates,
            'actual_direction': y_test_direction,
            'actual_return': y_test_df['next_log_return'].values,
            'take_profit_price': y_test_df['take_profit_price'].values,
            'stop_loss_price': y_test_df['stop_loss_price'].values,
            'pred_direction': pred_direction,
            'prob_neutral': prob_neutral,
            'prob_long': prob_long,
            'prob_short': prob_short,
            'trade_confidence': trade_confidence,
            'is_correct': (pred_direction == y_test_direction).astype(int)
        })
        
        self.predictions[model_name] = predictions_df
    
    def get_summary_dataframe(self):
        return pd.DataFrame(self.results)
    
    def get_predictions_dict(self):
        return self.predictions
    
    def get_models_dict(self):
        return self.models if self.models is not None else {}


class ModelTrainer:
    def __init__(self, evaluator, lookback=30):
        self.evaluator = evaluator
        self.lookback = lookback
    
    @staticmethod
    def create_sequences(X, y, lookback):
        Xs, ys = [], []
        for i in range(lookback, len(X)):
            Xs.append(X[i-lookback:i])
            ys.append(y.iloc[i] if hasattr(y, 'iloc') else y[i])
        X_arr = np.array(Xs)
        y_arr = np.array(ys)
        del Xs, ys
        gc.collect()
        return X_arr, y_arr
    
    @staticmethod
    def clear_memory():
        tf.keras.backend.clear_session()
        try:
            tf.compat.v1.reset_default_graph()
        except:
            pass
        gc.collect()
    
    def train_ml_model(self, model_config, X_train, y_train, X_val, y_val, X_test, y_test_df, test_dates, task='classification'):
        model = None
        try:
            if model_config.get('needs_val', False):
                model = model_config['func'](X_train, y_train, X_val, y_val)
            else:
                model = model_config['func'](X_train, y_train)
            
            is_mlp = (model_config['name'] == 'MLP')
            
            self.evaluator.evaluate_classification_model(
                model, X_train, y_train, X_val, y_val, X_test, y_test_df,
                test_dates, model_config['name'], is_deep_learning=is_mlp
            )
            
            if not self.evaluator.save_models:
                del model
                model = None
                if is_mlp:
                    self.clear_memory()
                else:
                    gc.collect()
            
            return True
        except Exception as e:
            print(f"    {model_config['name']} failed: {type(e).__name__}")
            import traceback
            traceback.print_exc()
            return False
        finally:
            if model is not None and not self.evaluator.save_models:
                try:
                    del model
                except:
                    pass
            if model_config.get('name') == 'MLP':
                self.clear_memory()
            else:
                gc.collect()
    
    def train_dl_model(self, model_config, X_train_seq, y_train_seq, X_val_seq, y_val_seq, X_test_seq, y_test_df_seq, test_dates_seq, input_shape, task='classification'):
        model = None
        try:
            self.clear_memory()
            
            model = model_config['func'](X_train_seq, y_train_seq, X_val_seq, y_val_seq, input_shape)
            
            self.evaluator.evaluate_classification_model(
                model, X_train_seq, y_train_seq, X_val_seq, y_val_seq,
                X_test_seq, y_test_df_seq, test_dates_seq,
                model_config['name'], is_deep_learning=True
            )
            
            if not self.evaluator.save_models:
                del model
                model = None
                self.clear_memory()
            
            return True
        except Exception as e:
            print(f"    {model_config['name']} failed: {type(e).__name__}")
            import traceback
            traceback.print_exc()
            return False
        finally:
            if model is not None and not self.evaluator.save_models:
                try:
                    del model
                except:
                    pass
            self.clear_memory()


def train_all_models(X_train, y_train, X_val, y_val, X_test, y_test_df, test_dates, evaluator, lookback=30, ml_models=None, dl_models=None):
    trainer = ModelTrainer(evaluator, lookback)

    ml_success = 0
    for model_config in ml_models:
        if trainer.train_ml_model(model_config, X_train, y_train, X_val, y_val, X_test, y_test_df, test_dates):
            ml_success += 1
        gc.collect()
    
    trainer.clear_memory()
    
    y_test_direction = y_test_df['next_direction'].values

    X_train_seq, y_train_seq = trainer.create_sequences(X_train, y_train, lookback)
    X_val_seq, y_val_seq = trainer.create_sequences(X_val, y_val, lookback)
    X_test_seq, y_test_seq = trainer.create_sequences(X_test, y_test_direction, lookback)
    
    test_dates_seq = test_dates[lookback:]
    y_test_df_seq = y_test_df.iloc[lookback:].reset_index(drop=True)
    
    input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
    
    dl_success = 0
    for model_config in dl_models:
        trainer.clear_memory()
        
        if model_config['name'] in ['TabNet', 'StackingEnsemble', 'VotingHard', 'VotingSoft']:
            if trainer.train_ml_model(model_config, X_train, y_train, X_val, y_val, X_test, y_test_df, test_dates):
                dl_success += 1
        else:
            if trainer.train_dl_model(model_config, X_train_seq, y_train_seq, X_val_seq, y_val_seq, X_test_seq, y_test_df_seq, test_dates_seq, input_shape):
                dl_success += 1
        
        gc.collect()
    
    del X_train_seq, y_train_seq, X_val_seq, y_val_seq, X_test_seq, y_test_seq, y_test_df_seq, test_dates_seq
    trainer.clear_memory()
    
    return ml_success + dl_success

In [8]:
def save_raw_data_once(pipeline_result, target_name, split_method):
    raw_dir = os.path.join(RESULT_DIR, "raw_data", target_name, split_method)
    os.makedirs(raw_dir, exist_ok=True)
    
    for fold_data in pipeline_result:
        fold_idx = fold_data['stats']['fold_idx']
        fold_type = fold_data['stats']['fold_type']
        fold_dir = os.path.join(raw_dir, f"fold_{fold_idx}_{fold_type}")
        os.makedirs(fold_dir, exist_ok=True)
        
        features = fold_data['stats']['selected_features']
        
        for split in ['train', 'val', 'test']:
            if 'X_raw' in fold_data[split]:
                df = pd.DataFrame(fold_data[split]['X_raw'], columns=features)
                df['date'] = fold_data[split]['dates']
                
                y_df = fold_data[split]['y']
                df = pd.concat([df, y_df], axis=1)
                
                df.to_csv(os.path.join(fold_dir, f"{split}_raw.csv"), index=False)

def check_fold_completed(target_name, fold_idx, fold_type):
    fold_dir = os.path.join(RESULT_DIR, "fold_results", target_name, f"fold_{fold_idx}_{fold_type}")
    
    if not os.path.isdir(fold_dir):
        return False
    
    required_files = [
        "fold_summary.csv",
        "robust_scaler.pkl",
        "standard_scaler.pkl",
        "selected_features.pkl",
        "inference_config.pkl"
    ]
    
    for file in required_files:
        if not os.path.exists(os.path.join(fold_dir, file)):
            return False
    
    model_files = [f for f in os.listdir(fold_dir) if f.endswith(('.pkl', '.h5')) 
                   and not any(x in f for x in ['scaler', 'features', 'stats', 'config'])]
    
    return len(model_files) > 0

def save_fold_results(fold_idx, fold_type, evaluator, target_name, fold_data):
    fold_dir = os.path.join(RESULT_DIR, "fold_results", target_name, f"fold_{fold_idx}_{fold_type}")
    os.makedirs(fold_dir, exist_ok=True)
    
    for model_name, model_obj in evaluator.get_models_dict().items():
        try:
            is_dl = isinstance(model_obj, tf.keras.Model)
            ext = ".h5" if is_dl else ".pkl"
            path = os.path.join(fold_dir, f"{model_name}{ext}")
            
            if is_dl: 
                model_obj.save(path)
            else:
                with open(path, 'wb') as f:
                    pickle.dump(model_obj, f)
        except Exception as e:
            print(f"Failed to save {model_name}: {e}")
    
    with open(os.path.join(fold_dir, "robust_scaler.pkl"), 'wb') as f:
        pickle.dump(fold_data['stats']['robust_scaler'], f)
    
    with open(os.path.join(fold_dir, "standard_scaler.pkl"), 'wb') as f:
        pickle.dump(fold_data['stats']['standard_scaler'], f)
    
    with open(os.path.join(fold_dir, "selected_features.pkl"), 'wb') as f:
        pickle.dump(fold_data['stats']['selected_features'], f)
    
    with open(os.path.join(fold_dir, "missing_stats.pkl"), 'wb') as f:
        pickle.dump(fold_data['stats']['missing_stats'], f)
    
    inference_config = {
        'selected_features': fold_data['stats']['selected_features'],
        'feature_order': fold_data['stats']['selected_features'], 
        'target_cols': fold_data['stats']['target_cols'],
        'target_type': fold_data['stats']['target_type'],
        'fold_type': fold_type,
        'fold_idx': fold_idx,
        'trend_params': {
            'trend_window': fold_data['stats'].get('trend_window', 120),
            'trend_analysis_points': fold_data['stats'].get('trend_analysis_points', 5)
        }
    }
    with open(os.path.join(fold_dir, "inference_config.pkl"), 'wb') as f:
        pickle.dump(inference_config, f)
    
    for model_name, pred_df in evaluator.get_predictions_dict().items():
        pred_df.to_csv(os.path.join(fold_dir, f"{model_name}_predictions.csv"), 
                       index=False, encoding='utf-8-sig')
        
    summary = evaluator.get_summary_dataframe()
    summary.to_csv(os.path.join(fold_dir, "fold_summary.csv"), 
                   index=False, encoding='utf-8-sig')
    
    return summary, evaluator.get_predictions_dict()

def load_fold_results(target_name, fold_idx, fold_type):
    fold_dir = os.path.join(RESULT_DIR, "fold_results", target_name, f"fold_{fold_idx}_{fold_type}")
    summary_path = os.path.join(fold_dir, "fold_summary.csv")
    if not os.path.exists(summary_path): 
        return None, None
    summary = pd.read_csv(summary_path)
    predictions = {f.replace('_predictions.csv', ''): pd.read_csv(os.path.join(fold_dir, f)) 
                   for f in os.listdir(fold_dir) if f.endswith('_predictions.csv')}
    return summary, predictions

def save_walk_forward_summary(all_fold_results, target_name):
    if not all_fold_results: 
        return
    detailed_df = pd.concat([df.assign(Fold=i+1, fold_type=ft) 
                             for i, (df, ft) in enumerate(all_fold_results)], ignore_index=True)
    detailed_df.to_csv(os.path.join(RESULT_DIR, f"{target_name}_all_folds_detailed.csv"), 
                       index=False, encoding='utf-8-sig')
    
    wf_data = detailed_df[detailed_df['fold_type'] == 'walk_forward_rolling'].copy()
    if wf_data.empty: 
        return

    numeric_cols = wf_data.select_dtypes(include=np.number).columns.drop('Fold', errors='ignore')
    avg_results = wf_data.groupby('Model')[numeric_cols].agg(['mean', 'std']).reset_index()
    avg_results.columns = ['_'.join(col).strip() if col[1] else col[0] for col in avg_results.columns.values]
    
    sort_col = 'Test_Precision_mean' if 'Test_Precision_mean' in avg_results.columns else 'Test_Accuracy_mean'
    avg_results = avg_results.sort_values(by=sort_col, ascending=False).reset_index(drop=True)
    
    avg_results.to_csv(os.path.join(RESULT_DIR, f"{target_name}_walk_forward_average.csv"), 
                       index=False, encoding='utf-8-sig')

def load_all_fold_results_from_disk(target_name):
    fold_results_dir = os.path.join(RESULT_DIR, "fold_results", target_name)
    if not os.path.isdir(fold_results_dir):
        return []
    
    fold_dirs = sorted([d for d in os.listdir(fold_results_dir) 
                        if os.path.isdir(os.path.join(fold_results_dir, d))])
    
    fold_results = []
    for fold_dir_name in fold_dirs:
        fold_parts = fold_dir_name.split('_')
        fold_idx = int(fold_parts[1])
        fold_type = '_'.join(fold_parts[2:])
        
        summary, _ = load_fold_results(target_name, fold_idx, fold_type)
        if summary is not None:
            fold_results.append((summary, fold_type))
    
    return fold_results

def run_and_save_master_summary(result_dir):
    summary_files = glob.glob(os.path.join(result_dir, "*_walk_forward_average.csv"))
    if not summary_files: 
        return

    master_list = []
    for f in summary_files:
        filename = os.path.basename(f)
        parts = filename.replace('trial_', '').replace('_walk_forward_average.csv', '').split('_')
        df = pd.read_csv(f)
        
        for p in parts:
            if p.startswith('l'):
                df['lookahead'] = int(p[1:])
            elif p.startswith('p'):
                df['profit_mult'] = float(p[1:])
            elif p.startswith('s'):
                df['stop_mult'] = float(p[1:])
            elif p.startswith('tw'):
                df['trend_window'] = int(p[2:])
            elif p.startswith('tap'):
                df['trend_analysis_points'] = int(p[3:])
        
        master_list.append(df)
        
    master_df = pd.concat(master_list, ignore_index=True)
    
    sort_col = 'Test_Precision_mean' if 'Test_Precision_mean' in master_df.columns else 'Test_Accuracy_mean'
    master_df = master_df.sort_values(by=sort_col, ascending=False).reset_index(drop=True)
    
    master_df.to_csv(os.path.join(result_dir, "_MASTER_SUMMARY_RESULTS.csv"), 
                     index=False, encoding='utf-8-sig')

def check_experiment_completed(target_name, result_dir):
    fold_results_dir = os.path.join(result_dir, "fold_results", target_name)
    if os.path.isdir(fold_results_dir):
        fold_dirs = [d for d in os.listdir(fold_results_dir) if os.path.isdir(os.path.join(fold_results_dir, d))]
        has_final_holdout = any('final_holdout' in d for d in fold_dirs)
        
        if has_final_holdout and len(fold_dirs) >= 2:
            summary_path = os.path.join(result_dir, f"{target_name}_walk_forward_average.csv")
            if os.path.exists(summary_path):
                return True
    
    return False

def record_optuna_result(trial_num, target_name, score, params):
    summary_file = os.path.join(RESULT_DIR, "optuna_trials_summary.csv")
    
    row = {
        'trial_number': trial_num,
        'target_name': target_name,
        'score': score,
        **params
    }
    
    df = pd.DataFrame([row])
    if os.path.exists(summary_file):
        df.to_csv(summary_file, mode='a', header=False, index=False)
    else:
        df.to_csv(summary_file, index=False)

In [9]:
# def objective(trial):
#     lookahead = trial.suggest_int('lookahead', 3, 5)
#     p_mult = trial.suggest_float('profit_mult', 0.8, 1.5, step=0.1)
#     s_mult = trial.suggest_float('stop_mult', 0.5, 1.0, step=0.1)
#     trend_window = trial.suggest_int('trend_window', 10, 30, step=5)
#     trend_analysis_points = trial.suggest_int('trend_analysis_points', 3, 5)
    
#     param_suffix = f"l{lookahead}_p{p_mult:.1f}_s{s_mult:.1f}_tw{trend_window}_tap{trend_analysis_points}"
#     trial_target_name = f"trial_{trial.number}_{param_suffix}"
    
#     if check_experiment_completed(trial_target_name, RESULT_DIR):
#         print(f"Trial {trial.number} already completed - skipping")
#         summary_path = os.path.join(RESULT_DIR, f"{trial_target_name}_walk_forward_average.csv")
#         existing_summary = pd.read_csv(summary_path)
        
#         if 'Test_Precision_mean' in existing_summary.columns:
#             return existing_summary['Test_Precision_mean'].max()
#         return 0.0
    
#     try:
#         pipeline_result = build_complete_pipeline_corrected(
#             df_raw=df_merged,
#             train_start_date=TRAIN_START_DATE,
#             final_test_start='2025-01-01',
#             method='walk_forward',
#             target_type='direction',
#             lookahead_candles=lookahead,
#             atr_multiplier_profit=p_mult,
#             atr_multiplier_stop=s_mult,
#             trend_window=trend_window,
#             trend_analysis_points=trend_analysis_points,
#             top_n=40
#         )
        
#         save_raw_data_once(pipeline_result, trial_target_name, 'walk_forward')
        
#         fold_results = []
        
#         for fold_data in pipeline_result:
#             fold_idx = fold_data['stats']['fold_idx']
#             fold_type = fold_data['stats']['fold_type']
            
#             if check_fold_completed(trial_target_name, fold_idx, fold_type):
#                 print(f"Fold {fold_idx} already completed")
#                 fold_summary, _ = load_fold_results(trial_target_name, fold_idx, fold_type)
#                 if fold_summary is not None:
#                     fold_results.append((fold_summary, fold_type))
#                 continue
            
#             print(f"Training Fold {fold_idx} ({fold_type})...")
            
#             evaluator = ModelEvaluator(save_models=True)
            
#             train_all_models(
#                 fold_data['train']['X_robust'], 
#                 fold_data['train']['y']['next_direction'].values.astype(int),
#                 fold_data['val']['X_robust'], 
#                 fold_data['val']['y']['next_direction'].values.astype(int),
#                 fold_data['test']['X_robust'], 
#                 fold_data['test']['y'],
#                 fold_data['test']['dates'].values,
#                 evaluator,
#                 ml_models=ML_MODELS_CLASSIFICATION,
#                 dl_models=DL_MODELS_CLASSIFICATION
#             )
            
#             fold_summary, _ = save_fold_results(fold_idx, fold_type, evaluator, trial_target_name, fold_data)
#             fold_results.append((fold_summary, fold_type))
            
#             del evaluator
#             tf.keras.backend.clear_session()
#             gc.collect()
        
#         save_walk_forward_summary(fold_results, trial_target_name)
        
#         summary_path = os.path.join(RESULT_DIR, f"{trial_target_name}_walk_forward_average.csv")
#         if os.path.exists(summary_path):
#             wf_avg = pd.read_csv(summary_path)
#             best_score = wf_avg['Test_Precision_mean'].max()
            
#             record_optuna_result(trial.number, trial_target_name, best_score, trial.params)
#             return best_score
        
#         return 0.0
        
#     except Exception as e:
#         print(f"Trial {trial.number} Failed: {e}")
#         traceback.print_exc()
#         return 0.0

# timestamp = datetime.now().strftime("%Y-%m-%d")
# RESULT_DIR = os.path.join("model_results", timestamp)
# os.makedirs(RESULT_DIR, exist_ok=True)

# if tf.config.list_physical_devices('GPU'):
#     try:
#         for gpu in tf.config.list_physical_devices('GPU'):
#             tf.config.experimental.set_memory_growth(gpu, True)
#     except RuntimeError as e:
#         print(e)

# study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
# study.optimize(objective, n_trials=20)

In [ ]:
import optuna
from optuna.samplers import TPESampler
import os
import pandas as pd
import tensorflow as tf
import traceback
import gc
from datetime import datetime

import optuna
from optuna.samplers import TPESampler
import os
import pandas as pd
import tensorflow as tf
import traceback
import gc
from datetime import datetime

TRIAL_START_OFFSET = 5

def objective(trial):
    lookahead = trial.suggest_int('lookahead', 3, 7)
    p_mult = trial.suggest_float('profit_mult', 0.8, 2.0, step=0.1)
    s_mult = trial.suggest_float('stop_mult', 0.4, 1.0, step=0.1)
    trend_window = trial.suggest_int('trend_window', 10, 40, step=5)
    trend_analysis_points = trial.suggest_int('trend_analysis_points', 3, 7)
    
    actual_trial_number = trial.number + TRIAL_START_OFFSET
    
    param_suffix = f"l{lookahead}_p{p_mult:.1f}_s{s_mult:.1f}_tw{trend_window}_tap{trend_analysis_points}"
    trial_target_name = f"trial_{actual_trial_number}_{param_suffix}"
    
    print(f"Processing: {trial_target_name} (Optuna Index: {trial.number})")
    
    if check_experiment_completed(trial_target_name, RESULT_DIR):
        print(f"Trial {actual_trial_number} already completed - skipping")
        summary_path = os.path.join(RESULT_DIR, f"{trial_target_name}_walk_forward_average.csv")
        if os.path.exists(summary_path):
            existing_summary = pd.read_csv(summary_path)
            if 'Test_Precision_mean' in existing_summary.columns:
                return existing_summary['Test_Precision_mean'].max()
        return 0.0
    
    try:
        pipeline_result = build_complete_pipeline_corrected(
            df_raw=df_merged,
            train_start_date=TRAIN_START_DATE,
            final_test_start='2025-01-01',
            method='walk_forward',
            target_type='direction',
            lookahead_candles=lookahead,
            atr_multiplier_profit=p_mult,
            atr_multiplier_stop=s_mult,
            trend_window=trend_window,
            trend_analysis_points=trend_analysis_points,
            top_n=20
        )
        
        save_raw_data_once(pipeline_result, trial_target_name, 'walk_forward')
        
        fold_results = []
        
        for fold_data in pipeline_result:
            fold_idx = fold_data['stats']['fold_idx']
            fold_type = fold_data['stats']['fold_type']
            
            if check_fold_completed(trial_target_name, fold_idx, fold_type):
                print(f"Fold {fold_idx} already completed in {trial_target_name}")
                fold_summary, _ = load_fold_results(trial_target_name, fold_idx, fold_type)
                if fold_summary is not None:
                    fold_results.append((fold_summary, fold_type))
                continue
            
            print(f"Training Fold {fold_idx} ({fold_type})...")
            
            evaluator = ModelEvaluator(save_models=True)
            
            train_all_models(
                fold_data['train']['X_robust'], 
                fold_data['train']['y']['next_direction'].values.astype(int),
                fold_data['val']['X_robust'], 
                fold_data['val']['y']['next_direction'].values.astype(int),
                fold_data['test']['X_robust'], 
                fold_data['test']['y'],
                fold_data['test']['dates'].values,
                evaluator,
                ml_models=ML_MODELS_CLASSIFICATION,
                dl_models=DL_MODELS_CLASSIFICATION
            )
            
            fold_summary, _ = save_fold_results(fold_idx, fold_type, evaluator, trial_target_name, fold_data)
            fold_results.append((fold_summary, fold_type))
            
            del evaluator
            tf.keras.backend.clear_session()
            gc.collect()
        
        save_walk_forward_summary(fold_results, trial_target_name)
        
        summary_path = os.path.join(RESULT_DIR, f"{trial_target_name}_walk_forward_average.csv")
        if os.path.exists(summary_path):
            wf_avg = pd.read_csv(summary_path)
            best_score = wf_avg['Test_Precision_mean'].max()
            
            record_optuna_result(actual_trial_number, trial_target_name, best_score, trial.params)
            return best_score
        
        return 0.0
        
    except Exception as e:
        print(f"Trial {actual_trial_number} Failed: {e}")
        traceback.print_exc()
        return 0.0

timestamp = datetime.now().strftime("%Y-%m-%d")
RESULT_DIR = os.path.join("model_results", timestamp)
os.makedirs(RESULT_DIR, exist_ok=True)

if tf.config.list_physical_devices('GPU'):
    try:
        for gpu in tf.config.list_physical_devices('GPU'):
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))

study.enqueue_trial({
    'lookahead': 5,
    'profit_mult': 1.2,
    'stop_mult': 0.7,
    'trend_window': 20,
    'trend_analysis_points': 4
})

study.optimize(objective, n_trials=30)

[I 2025-11-19 23:25:04,111] A new study created in memory with name: no-name-0dc0e4a6-ce5d-4b42-8dc3-ffacb2dc5913


Processing: trial_5_l5_p1.2_s0.7_tw20_tap4 (Optuna Index: 0)

--- Fold 1 (walk_forward_rolling) Class Balance Analysis (top_n=20) ---
 Train Set (N=800): Hold(0): 14.12% | Long(1): 32.88% | Short(2): 53.00%
 Validation Set (N=150): Hold(0): 19.33% | Long(1): 25.33% | Short(2): 55.33%
 Test Set (N=150): Hold(0): 13.33% | Long(1): 34.00% | Short(2): 52.67%
------------------------------------------------------------
ATR_14, DPO_20, DMP_14, DMN_14, btc_eth_strength_ratio_7d, sp500_SP500_pct_5d_lag1, vix_VIX_pct_5d_lag1, gold_GOLD_pct_5d_lag1, btc_volatility_7d, VWAP, ROLLING_MIN_20, ISB_26, SMA_50, IKS_26, ISA_9, ROLLING_MAX_20, VWMA_20, BBL_20, AD, WMA_20

--- Fold 2 (walk_forward_rolling) Class Balance Analysis (top_n=20) ---
 Train Set (N=800): Hold(0): 14.75% | Long(1): 31.25% | Short(2): 54.00%
 Validation Set (N=150): Hold(0): 14.67% | Long(1): 32.00% | Short(2): 53.33%
 Test Set (N=150): Hold(0): 18.67% | Long(1): 24.00% | Short(2): 57.33%
------------------------------------------

2025-11-19 23:28:12.314358: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46227 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:1d:00.0, compute capability: 8.6
2025-11-19 23:28:15.954840: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2025-11-19 23:28:17.182058: I external/local_xla/xla/service/service.cc:168] XLA service 0x3fc3f290 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-19 23:28:17.182100: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2025-11-19 23:28:17.188457: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1763562497.298098  383758 device_compiler.h:186] Compiled cluster using XLA!  This line is logg

[LSTM] Acc: 0.6416/0.6083 (Gap: 0.0332)
4/4 [==============================] - 0s 2ms/step
[LSTM] Total Acc: 0.5750 | Long(1) Prec: 0.4833 | Short(2) Prec: 0.6667
[BiLSTM] Acc: 0.6506/0.5167 (Gap: 0.1340)
4/4 [==============================] - 0s 3ms/step
[BiLSTM] Total Acc: 0.6167 | Long(1) Prec: 0.5263 | Short(2) Prec: 0.7692
[GRU] Acc: 0.6143/0.5750 (Gap: 0.0393)
4/4 [==============================] - 0s 2ms/step
[GRU] Total Acc: 0.5583 | Long(1) Prec: 0.4444 | Short(2) Prec: 0.7949
Training Fold 2 (walk_forward_rolling)...
[CatBoost] Acc: 0.6212/0.6200 (Gap: 0.0012)
[CatBoost] Total Acc: 0.5733 | Long(1) Prec: 0.3714 | Short(2) Prec: 0.7595
[RandomForest] Acc: 0.7588/0.6333 (Gap: 0.1254)
[RandomForest] Total Acc: 0.6200 | Long(1) Prec: 0.4423 | Short(2) Prec: 0.7143
[LightGBM] Acc: 0.7625/0.6600 (Gap: 0.1025)
[LightGBM] Total Acc: 0.5933 | Long(1) Prec: 0.4098 | Short(2) Prec: 0.7273
[XGBoost] Acc: 0.9688/0.5800 (Gap: 0.3888)
[XGBoost] Total Acc: 0.5000 | Long(1) Prec: 0.3455 | Sho

Training Fold 6 (final_holdout)...
[CatBoost] Acc: 0.7800/0.5333 (Gap: 0.2467)
[CatBoost] Total Acc: 0.5478 | Long(1) Prec: 0.5652 | Short(2) Prec: 0.7456
[RandomForest] Acc: 0.8688/0.5800 (Gap: 0.2888)
[RandomForest] Total Acc: 0.5955 | Long(1) Prec: 0.5750 | Short(2) Prec: 0.6398
[LightGBM] Acc: 0.9988/0.5600 (Gap: 0.4387)
[LightGBM] Total Acc: 0.6083 | Long(1) Prec: 0.5775 | Short(2) Prec: 0.7105
[XGBoost] Acc: 0.9700/0.5467 (Gap: 0.4233)
[XGBoost] Total Acc: 0.5860 | Long(1) Prec: 0.5283 | Short(2) Prec: 0.6747
[GradientBoost] Acc: 0.9750/0.5467 (Gap: 0.4283)
[GradientBoosting] Total Acc: 0.5255 | Long(1) Prec: 0.4706 | Short(2) Prec: 0.6095
[HistGradient] Acc: 0.8000/0.5000 (Gap: 0.3000)
[HistGradientBoosting] Total Acc: 0.5223 | Long(1) Prec: 0.5455 | Short(2) Prec: 0.6864
[LogisticReg] Acc: 0.6112/0.4267 (Gap: 0.1846)
[LogisticRegression] Total Acc: 0.3790 | Long(1) Prec: 0.7931 | Short(2) Prec: 0.8824
[Stacking] Acc: 0.7325/0.5467 (Gap: 0.1858)
[StackingEnsemble] Total Acc: 0.5

Traceback (most recent call last):
  File "/raid/invigoworks/anaconda3/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3802, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Test_Precision_mean'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/tmp/ipykernel_381508/2192209421.py", line 103, in objective
    best_score = wf_avg['Test_Precision_mean'].max()
  File "/raid/invigoworks/anaconda3/lib/python3.10/site-packages/pandas/core/frame.py", line 3807, in __getitem__
    indexer = self.columns.get_l


--- Fold 1 (walk_forward_rolling) Class Balance Analysis (top_n=20) ---
 Train Set (N=800): Hold(0): 41.88% | Long(1): 14.62% | Short(2): 43.50%
 Validation Set (N=150): Hold(0): 49.33% | Long(1): 11.33% | Short(2): 39.33%
 Test Set (N=150): Hold(0): 42.67% | Long(1): 20.67% | Short(2): 36.67%
------------------------------------------------------------
ATR_14, DPO_20, EMA_26, MOM_10, eth_btc_volcorr_sq_14d_pct_5d, btc_volatility_7d, VWAP, SMA_50, IKS_26, ROLLING_MIN_20, ITS_9, VWMA_20, ISB_26, KCB_20, SMA_20, WMA_20, ISA_9, eth_btc_corr_highvol, KCU_20, AD

--- Fold 2 (walk_forward_rolling) Class Balance Analysis (top_n=20) ---
 Train Set (N=800): Hold(0): 43.88% | Long(1): 12.62% | Short(2): 43.50%
 Validation Set (N=150): Hold(0): 44.67% | Long(1): 20.00% | Short(2): 35.33%
 Test Set (N=150): Hold(0): 42.67% | Long(1): 14.67% | Short(2): 42.67%
------------------------------------------------------------
ATR_14, DPO_20, eth_btc_corr_highvol, xrp_volatility_30d, MOM_10, eth_btc_corr

[GRU] Acc: 0.6351/0.5417 (Gap: 0.0934)
4/4 [==============================] - 0s 2ms/step
[GRU] Total Acc: 0.4500 | Long(1) Prec: 0.5000 | Short(2) Prec: 0.4464
Training Fold 3 (walk_forward_rolling)...
[CatBoost] Acc: 0.7338/0.5467 (Gap: 0.1871)
[CatBoost] Total Acc: 0.5667 | Long(1) Prec: 0.5000 | Short(2) Prec: 0.6304
[RandomForest] Acc: 0.9062/0.4867 (Gap: 0.4196)
[RandomForest] Total Acc: 0.5933 | Long(1) Prec: 0.5926 | Short(2) Prec: 0.5965
[LightGBM] Acc: 0.9175/0.4933 (Gap: 0.4242)
[LightGBM] Total Acc: 0.5067 | Long(1) Prec: 0.5323 | Short(2) Prec: 0.5132
[XGBoost] Acc: 0.7588/0.4800 (Gap: 0.2788)
[XGBoost] Total Acc: 0.5267 | Long(1) Prec: 0.5000 | Short(2) Prec: 0.5469
[GradientBoost] Acc: 0.7050/0.5133 (Gap: 0.1917)
[GradientBoosting] Total Acc: 0.5400 | Long(1) Prec: 0.4615 | Short(2) Prec: 0.6000
[HistGradient] Acc: 0.7412/0.4800 (Gap: 0.2612)
[HistGradientBoosting] Total Acc: 0.5267 | Long(1) Prec: 0.4930 | Short(2) Prec: 0.5469
[LogisticReg] Acc: 0.6362/0.4533 (Gap: 0.1